# 정원 CSV

In [ ]:
import sqlite3
import csv

# 1. SQLite 데이터베이스에 연결
# 'samsung_audit_report.db'는 원하는 데이터베이스 파일명으로 변경할 수 있습니다.
conn = sqlite3.connect('samsung_audit_report.db')
cursor = conn.cursor()

# 2. 재고자산_내역 테이블 생성 (이미 존재하면 건너뜁니다)
# 이전에 제공해드린 쿼리를 다시 포함하여 코드를 한 번에 실행할 수 있게 했습니다.
create_table_query = """
CREATE TABLE IF NOT EXISTS 재고자산_내역 (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    연도 INTEGER NOT NULL,
    구분 TEXT NOT NULL,
    당기말평가전금액 REAL,
    당기말평가충당금 REAL,
    당기말장부가액 REAL,
    단위 TEXT
);
"""
cursor.execute(create_table_query)
conn.commit()

# 3. CSV 데이터 삽입
# CSV 파일 경로 (업로드한 파일명을 정확히 사용해야 합니다)
csv_file_path = 'DB_csv/전체_재고자산_내역.csv'

try:
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # 헤더(첫 번째 행) 건너뛰기

        # 각 행의 데이터를 테이블에 삽입
        for row in csv_reader:
            # 금액 데이터에서 쉼표(,)와 따옴표를 제거하고 실수형(REAL)으로 변환
            year = int(row[0])
            category = row[1]
            amount_before_valuation = float(row[2].replace(',', '').strip('"'))
            # '당기말평가충당금'이 '-'인 경우 NULL 처리
            valuation_allowance_str = row[3].replace(',', '').strip('"')
            valuation_allowance = None if valuation_allowance_str == '-' else float(valuation_allowance_str)
            book_value = float(row[4].replace(',', '').strip('"'))
            unit = row[5]

            # 데이터 삽입을 위한 SQL 명령
            insert_query = """
            INSERT INTO 재고자산_내역 (
                연도,
                구분,
                당기말평가전금액,
                당기말평가충당금,
                당기말장부가액,
                단위
            ) VALUES (?, ?, ?, ?, ?, ?);
            """
            cursor.execute(insert_query, (
                year,
                category,
                amount_before_valuation,
                valuation_allowance,
                book_value,
                unit
            ))

    # 변경사항 커밋
    conn.commit()
    print("CSV 데이터가 데이터베이스에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"오류: {csv_file_path} 파일을 찾을 수 없습니다. 파일 경로를 확인해 주세요.")

finally:
    # 4. 데이터베이스 연결 종료
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

CSV 데이터가 데이터베이스에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [ ]:
import sqlite3
import csv

# 1. SQLite 데이터베이스에 연결
conn = sqlite3.connect('samsung_audit_report.db')
cursor = conn.cursor()

# 2. 종속기업, 관계기업 및 공동기업 투자 변동내역 테이블 생성
create_table_query = """
CREATE TABLE IF NOT EXISTS 종속기업_관계기업_공동기업_투자_변동내역 (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    연도 INTEGER NOT NULL,
    구분 TEXT NOT NULL,
    변동내역 REAL,
    단위 TEXT
);
"""
cursor.execute(create_table_query)
conn.commit()

# 3. CSV 데이터 삽입
csv_file_path = 'DB_csv/전체_종속기업,관계기업및공동기업투자_변동내역.csv'

try:
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # 헤더(첫 번째 행) 건너뛰기

        # 각 행의 데이터를 테이블에 삽입
        for row in csv_reader:
            year = int(row[0])
            category = row[1]
            
            # '변동내역' 금액에서 쉼표(,)와 따옴표를 제거하고, '-'인 경우 NULL 처리
            change_value_str = row[2].replace(',', '').strip('"')
            change_value = None if change_value_str == '-' else float(change_value_str)
            
            unit = row[3]

            # 데이터 삽입을 위한 SQL 명령
            insert_query = """
            INSERT INTO 종속기업_관계기업_공동기업_투자_변동내역 (
                연도,
                구분,
                변동내역,
                단위
            ) VALUES (?, ?, ?, ?);
            """
            cursor.execute(insert_query, (
                year,
                category,
                change_value,
                unit
            ))

    # 변경사항 커밋
    conn.commit()
    print("CSV 데이터가 데이터베이스에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"오류: {csv_file_path} 파일을 찾을 수 없습니다. 파일 경로를 확인해 주세요.")

finally:
    # 4. 데이터베이스 연결 종료
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

CSV 데이터가 데이터베이스에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [ ]:
import sqlite3
import csv
import re

# 1. SQLite 데이터베이스에 연결
conn = sqlite3.connect('samsung_audit_report.db')
cursor = conn.cursor()

# 2. '종속기업_재무정보' 테이블 생성
create_table_query = """
CREATE TABLE IF NOT EXISTS 종속기업_재무정보 (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    연도 INTEGER NOT NULL,
    기업명 TEXT NOT NULL,
    자산 REAL,
    부채 REAL,
    매출액 REAL,
    당기순이익 REAL,
    단위 TEXT
);
"""
cursor.execute(create_table_query)
conn.commit()

# 3. CSV 데이터 삽입
csv_file_path = 'DB_csv/전체_종속기업_재무정보.csv'

def clean_numeric_value(value):
    """숫자 문자열에서 쉼표와 따옴표를 제거하고, 'nan'을 None으로 변환합니다."""
    if value.strip().lower() == 'nan':
        return None
    # 따옴표와 쉼표를 제거하고 실수형으로 변환
    cleaned_value = re.sub(r'[",]', '', value)
    return float(cleaned_value)

try:
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # 헤더(첫 번째 행) 건너뛰기

        for row in csv_reader:
            # 데이터 정리 및 타입 변환
            year = int(row[0])
            company_name = row[1]
            assets = clean_numeric_value(row[2])
            liabilities = clean_numeric_value(row[3])
            sales = clean_numeric_value(row[4])
            net_income = clean_numeric_value(row[5])
            unit = row[6]

            # 데이터 삽입을 위한 SQL 명령
            insert_query = """
            INSERT INTO 종속기업_재무정보 (
                연도,
                기업명,
                자산,
                부채,
                매출액,
                당기순이익,
                단위
            ) VALUES (?, ?, ?, ?, ?, ?, ?);
            """
            cursor.execute(insert_query, (
                year,
                company_name,
                assets,
                liabilities,
                sales,
                net_income,
                unit
            ))

    # 변경사항 커밋
    conn.commit()
    print("CSV 데이터가 데이터베이스에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"오류: {csv_file_path} 파일을 찾을 수 없습니다. 파일 경로를 확인해 주세요.")

finally:
    # 4. 데이터베이스 연결 종료
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

CSV 데이터가 데이터베이스에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [ ]:
import sqlite3
import csv
import re

# 1. SQLite 데이터베이스에 연결
conn = sqlite3.connect('samsung_audit_report.db')
cursor = conn.cursor()

# 2. '유형자산_변동내역' 테이블 생성
create_table_query = """
CREATE TABLE IF NOT EXISTS 유형자산_변동내역 (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    연도 INTEGER NOT NULL,
    구분 TEXT NOT NULL,
    토지 REAL,
    건물및구축물 REAL,
    기계장치 REAL,
    건설중인자산 REAL,
    기타 REAL,
    계 REAL,
    단위 TEXT
);
"""
cursor.execute(create_table_query)
conn.commit()

# 3. CSV 데이터 삽입
csv_file_path = 'DB_csv/전체_유형자산_변동내역.csv'

def clean_numeric_value(value):
    """숫자 문자열에서 쉼표와 따옴표를 제거하고, 빈 값을 None으로 변환합니다."""
    value = value.strip()
    if not value:
        return None
    # 따옴표와 쉼표를 제거하고 실수형으로 변환
    cleaned_value = re.sub(r'[",]', '', value)
    return float(cleaned_value)

try:
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # 헤더(첫 번째 행) 건너뛰기

        for row in csv_reader:
            # 첫 번째 빈 컬럼을 제외하고 데이터 정리 및 타입 변환
            # CSV 파일 첫 컬럼은 비어있으므로 row[0]부터가 아닌 row[1]부터 시작
            year = int(row[1])
            category = row[2]
            
            land = clean_numeric_value(row[3])
            buildings = clean_numeric_value(row[4])
            machinery = clean_numeric_value(row[5])
            assets_under_construction = clean_numeric_value(row[6])
            others = clean_numeric_value(row[7])
            total = clean_numeric_value(row[8])
            unit = row[9]

            # 데이터 삽입을 위한 SQL 명령
            insert_query = """
            INSERT INTO 유형자산_변동내역 (
                연도,
                구분,
                토지,
                건물및구축물,
                기계장치,
                건설중인자산,
                기타,
                계,
                단위
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?);
            """
            cursor.execute(insert_query, (
                year,
                category,
                land,
                buildings,
                machinery,
                assets_under_construction,
                others,
                total,
                unit
            ))

    # 변경사항 커밋
    conn.commit()
    print("CSV 데이터가 데이터베이스에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"오류: {csv_file_path} 파일을 찾을 수 없습니다. 파일 경로를 확인해 주세요.")

finally:
    # 4. 데이터베이스 연결 종료
    cursor.close

CSV 데이터가 데이터베이스에 성공적으로 삽입되었습니다.


In [ ]:
import sqlite3
import csv
import re

# 1. SQLite 데이터베이스에 연결
conn = sqlite3.connect('samsung_audit_report.db')
cursor = conn.cursor()

# 2. '무형자산_변동내역' 테이블 생성
create_table_query = """
CREATE TABLE IF NOT EXISTS 무형자산_변동내역 (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    연도 INTEGER NOT NULL,
    구분 TEXT NOT NULL,
    산업재산권 REAL,
    개발비 REAL,
    회원권 REAL,
    영업권 REAL,
    기타의무형자산 REAL,
    계 REAL,
    단위 TEXT,
    기타 REAL
);
"""
cursor.execute(create_table_query)
conn.commit()

# 3. CSV 데이터 삽입
csv_file_path = 'DB_csv/전체_무형자산_변동내역.csv'

def clean_numeric_value(value):
    """숫자 문자열에서 쉼표와 따옴표를 제거하고, 빈 값을 None으로 변환합니다."""
    value = value.strip()
    if not value:
        return None
    # 따옴표와 쉼표를 제거하고 실수형으로 변환
    cleaned_value = re.sub(r'[",]', '', value)
    return float(cleaned_value)

try:
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # 헤더(첫 번째 행) 건너뛰기

        for row in csv_reader:
            # 첫 번째 빈 컬럼을 제외하고 데이터 정리 및 타입 변환
            # CSV 파일 첫 컬럼은 비어있으므로 row[0]부터가 아닌 row[1]부터 시작
            year = int(row[1])
            category = row[2]
            
            # 숫자 데이터 정리
            intellectual_property = clean_numeric_value(row[3])
            development_cost = clean_numeric_value(row[4])
            membership_rights = clean_numeric_value(row[5])
            goodwill = clean_numeric_value(row[6])
            other_intangibles = clean_numeric_value(row[7])
            total = clean_numeric_value(row[8])
            
            unit = row[9]
            
            # 마지막 기타 컬럼 처리
            # 빈 값은 None으로, 숫자는 실수형으로 변환
            other_value = clean_numeric_value(row[10])

            # 데이터 삽입을 위한 SQL 명령
            insert_query = """
            INSERT INTO 무형자산_변동내역 (
                연도,
                구분,
                산업재산권,
                개발비,
                회원권,
                영업권,
                기타의무형자산,
                계,
                단위,
                기타
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
            """
            cursor.execute(insert_query, (
                year,
                category,
                intellectual_property,
                development_cost,
                membership_rights,
                goodwill,
                other_intangibles,
                total,
                unit,
                other_value
            ))

    # 변경사항 커밋
    conn.commit()
    print("CSV 데이터가 데이터베이스에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"오류: {csv_file_path} 파일을 찾을 수 없습니다. 파일 경로를 확인해 주세요.")

finally:
    # 4. 데이터베이스 연결 종료
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

CSV 데이터가 데이터베이스에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [ ]:
import sqlite3
import csv
import re

# 1. SQLite 데이터베이스에 연결
conn = sqlite3.connect('samsung_audit_report.db')
cursor = conn.cursor()

# 2. '관계기업_투자주식_내역' 테이블 생성
create_table_query = """
CREATE TABLE IF NOT EXISTS 관계기업_투자주식_내역 (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    연도 INTEGER NOT NULL,
    구분 TEXT NOT NULL,
    주식수 REAL,
    시장가치 REAL,
    장부금액 REAL,
    단위 TEXT
);
"""
cursor.execute(create_table_query)
conn.commit()

# 3. CSV 데이터 삽입
csv_file_path = 'DB_csv/전체_관계기업_투자주식_내역.csv'

def clean_numeric_value(value):
    """숫자 문자열에서 쉼표와 따옴표를 제거하고, 빈 값 또는 '-'를 None으로 변환합니다."""
    value = value.strip()
    if not value or value == '-':
        return None
    # 따옴표와 쉼표를 제거하고 실수형으로 변환
    cleaned_value = re.sub(r'[",]', '', value)
    return float(cleaned_value)

try:
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # 헤더(첫 번째 행) 건너뛰기

        for row in csv_reader:
            # 첫 번째 빈 컬럼을 제외하고 데이터 정리 및 타입 변환
            # CSV 파일 첫 컬럼은 비어있으므로 row[0]부터가 아닌 row[1]부터 시작
            year = int(row[1])
            category = row[2]
            
            # 숫자 데이터 정리
            num_shares = clean_numeric_value(row[3])
            market_value = clean_numeric_value(row[4])
            book_value = clean_numeric_value(row[5])
            
            unit = row[6]

            # 데이터 삽입을 위한 SQL 명령
            insert_query = """
            INSERT INTO 관계기업_투자주식_내역 (
                연도,
                구분,
                주식수,
                시장가치,
                장부금액,
                단위
            ) VALUES (?, ?, ?, ?, ?, ?);
            """
            cursor.execute(insert_query, (
                year,
                category,
                num_shares,
                market_value,
                book_value,
                unit
            ))

    # 변경사항 커밋
    conn.commit()
    print("CSV 데이터가 데이터베이스에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"오류: {csv_file_path} 파일을 찾을 수 없습니다. 파일 경로를 확인해 주세요.")

finally:
    # 4. 데이터베이스 연결 종료
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

CSV 데이터가 데이터베이스에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [ ]:
import sqlite3
import csv
import re

# 1. SQLite 데이터베이스에 연결
conn = sqlite3.connect('samsung_audit_report.db')
cursor = conn.cursor()

# 2. '관계기업_투자_현황' 테이블 생성
create_table_query = """
CREATE TABLE IF NOT EXISTS 관계기업_투자_현황 (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    연도 INTEGER NOT NULL,
    기업명 TEXT NOT NULL,
    관계의성격 TEXT,
    지분율 REAL,
    주사업장 TEXT
);
"""
cursor.execute(create_table_query)
conn.commit()

# 3. CSV 데이터 삽입
csv_file_path = 'DB_csv/전체_관계기업_투자_현황.csv'

def clean_percentage_value(value):
    """지분율 문자열에서 '%'를 제거하고, 실수형으로 변환합니다."""
    value = value.strip()
    if not value or value == '-':
        return None
    # '%'를 제거하고 실수형으로 변환
    cleaned_value = value.replace('%', '')
    return float(cleaned_value)

try:
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # 헤더(첫 번째 행) 건너뛰기

        for row in csv_reader:
            year = int(row[0])
            company_name = row[1]
            nature_of_relationship = row[2]
            
            # 지분율 데이터 정리
            equity_ratio = clean_percentage_value(row[3])
            
            main_business_location = row[4]

            # 데이터 삽입을 위한 SQL 명령
            insert_query = """
            INSERT INTO 관계기업_투자_현황 (
                연도,
                기업명,
                관계의성격,
                지분율,
                주사업장
            ) VALUES (?, ?, ?, ?, ?);
            """
            cursor.execute(insert_query, (
                year,
                company_name,
                nature_of_relationship,
                equity_ratio,
                main_business_location
            ))

    # 변경사항 커밋
    conn.commit()
    print("CSV 데이터가 데이터베이스에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"오류: {csv_file_path} 파일을 찾을 수 없습니다. 파일 경로를 확인해 주세요.")

finally:
    # 4. 데이터베이스 연결 종료
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

CSV 데이터가 데이터베이스에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [ ]:
import sqlite3
import csv
import re

# 1. SQLite 데이터베이스에 연결
conn = sqlite3.connect('samsung_audit_report.db')
cursor = conn.cursor()

# 2. '관계기업_재무정보' 테이블 생성
create_table_query = """
CREATE TABLE IF NOT EXISTS 관계기업_재무정보 (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    기업명 TEXT NOT NULL,
    연도 INTEGER NOT NULL,
    유동자산 REAL,
    비유동자산 REAL,
    유동부채 REAL,
    비유동부채 REAL,
    순자산 REAL,
    매출 REAL,
    계속영업손익 REAL,
    기타포괄손익 REAL,
    총포괄손익 REAL,
    단위 TEXT,
    세후중단영업이익 REAL,
    세후중단영업손익 REAL
);
"""
cursor.execute(create_table_query)
conn.commit()

# 3. CSV 데이터 삽입
csv_file_path = 'DB_csv/전체_관계기업_재무정보.csv'

def clean_numeric_value(value):
    """숫자 문자열에서 쉼표, 따옴표를 제거하고, 빈 값 또는 'nan', 'None', '-'를 None으로 변환합니다."""
    value = value.strip()
    if not value or value.lower() in ('nan', 'none') or value == '-':
        return None
    # 따옴표와 쉼표를 제거하고 실수형으로 변환
    cleaned_value = re.sub(r'[",]', '', value)
    return float(cleaned_value)

try:
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # 헤더(첫 번째 행) 건너뛰기

        for row in csv_reader:
            # 첫 번째 컬럼은 기업명, 두 번째부터 연도와 재무 정보
            company_name = row[0]
            year = int(row[1])
            
            # 숫자 데이터 정리
            current_assets = clean_numeric_value(row[2])
            non_current_assets = clean_numeric_value(row[3])
            current_liabilities = clean_numeric_value(row[4])
            non_current_liabilities = clean_numeric_value(row[5])
            net_assets = clean_numeric_value(row[6])
            sales = clean_numeric_value(row[7])
            income_from_continuing_operations = clean_numeric_value(row[8])
            other_comprehensive_income = clean_numeric_value(row[9])
            total_comprehensive_income = clean_numeric_value(row[10])
            unit = row[11]
            discontinued_operations_income = clean_numeric_value(row[12])
            discontinued_operations_loss = clean_numeric_value(row[13])

            # 데이터 삽입을 위한 SQL 명령
            insert_query = """
            INSERT INTO 관계기업_재무정보 (
                기업명, 연도, 유동자산, 비유동자산, 유동부채, 비유동부채, 순자산, 매출,
                계속영업손익, 기타포괄손익, 총포괄손익, 단위, 세후중단영업이익, 세후중단영업손익
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
            """
            cursor.execute(insert_query, (
                company_name, year, current_assets, non_current_assets,
                current_liabilities, non_current_liabilities, net_assets,
                sales, income_from_continuing_operations,
                other_comprehensive_income, total_comprehensive_income,
                unit, discontinued_operations_income,
                discontinued_operations_loss
            ))

    # 변경사항 커밋
    conn.commit()
    print("CSV 데이터가 데이터베이스에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"오류: {csv_file_path} 파일을 찾을 수 없습니다. 파일 경로를 확인해 주세요.")

finally:
    # 4. 데이터베이스 연결 종료
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

CSV 데이터가 데이터베이스에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [17]:
import sqlite3

# 확인하고 싶은 테이블 목록
tables = [
    '재고자산_내역',
    '종속기업_관계기업_공동기업_투자_변동내역',
    '종속기업_재무정보',
    '유형자산_변동내역',
    '무형자산_변동내역',
    '관계기업_투자주식_내역',
    '관계기업_투자_현황',
    '관계기업_재무정보'
]

# 데이터베이스에 연결
conn = sqlite3.connect('samsung_audit_report.db')
cursor = conn.cursor()

try:
    print("=== 데이터베이스 테이블 확인 ===")
    for table_name in tables:
        print(f"\n--- 테이블: {table_name} ---")
        
        # 총 행의 개수 확인
        count_query = f"SELECT COUNT(*) FROM {table_name};"
        cursor.execute(count_query)
        count = cursor.fetchone()[0]
        print(f"총 데이터 개수: {count}개")
        
        # 상위 5개 행 조회
        select_query = f"SELECT * FROM {table_name} LIMIT 5;"
        cursor.execute(select_query)
        rows = cursor.fetchall()

        if rows:
            # 컬럼 이름 가져오기
            columns = [description[0] for description in cursor.description]
            print(f"컬럼: {', '.join(columns)}")
            
            # 데이터 출력
            for row in rows:
                print(row)
        else:
            print("데이터가 없습니다.")

except sqlite3.OperationalError as e:
    print(f"오류: {e}")
    print("테이블 이름이 올바르지 않거나, 테이블이 존재하지 않습니다.")

finally:
    # 연결 종료
    cursor.close()
    conn.close()
    print("\n=== 확인 완료 ===")

=== 데이터베이스 테이블 확인 ===

--- 테이블: 재고자산_내역 ---
총 데이터 개수: 55개
컬럼: id, 연도, 구분, 당기말평가전금액, 당기말평가충당금, 당기말장부가액, 단위
(1, 2014, '제품 및 상품', 1003347.0, 61090.0, 942257.0, '백만원')
(2, 2014, '반제품 및 재공품', 3544921.0, 364146.0, 3180775.0, '백만원')
(3, 2014, '원재료 및 저장품', 1371893.0, 226386.0, 1145507.0, '백만원')
(4, 2014, '미착품', 285295.0, None, 285295.0, '백만원')
(5, 2014, '계', 6205456.0, 651622.0, 5553834.0, '백만원')

--- 테이블: 종속기업_관계기업_공동기업_투자_변동내역 ---
총 데이터 개수: 59개
컬럼: id, 연도, 구분, 변동내역, 단위
(1, 2014, '기초장부가액', 42258049.0, '백만원')
(2, 2014, '취득', 1236303.0, '백만원')
(3, 2014, '처분', 48869.0, '백만원')
(4, 2014, '손상', 82616.0, '백만원')
(5, 2014, '기타변동액', 1505436.0, '백만원')

--- 테이블: 종속기업_재무정보 ---
총 데이터 개수: 165개
컬럼: id, 연도, 기업명, 자산, 부채, 매출액, 당기순이익, 단위
(1, 2014, '삼성디스플레이', 37939965.0, 6728991.0, 24980628.0, 1698239.0, '백만원')
(2, 2014, 'Samsung Electronics AmericaSEA', 10697787.0, 5409749.0, 14076007.0, None, '백만원')
(3, 2014, 'Samsung China InvestmentSCIC', 9603679.0, 7830941.0, 16451983.0, 162677.0, '백만원')
(4, 2014, 'Samsung E

# 연우 CSV

In [59]:
import sqlite3
import csv

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/[환율변동위험]matrix_long_raw.csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

try:
    # Create a new table for the foreign exchange risk data
    create_table_query = """
    CREATE TABLE IF NOT EXISTS 환율변동위험 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        연도 INTEGER NOT NULL,
        항목 TEXT,
        변동내역 TEXT,
        금액 REAL,
        단위 TEXT,
        단위_배수 INTEGER
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 '환율변동위험'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO 환율변동위험 (
            연도,
            항목,
            변동내역,
            금액,
            단위,
            단위_배수
        ) VALUES (?, ?, ?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                year = int(row[0])
                line_item = row[1]
                column_name = row[2]
                amount_raw = float(row[3])
                unit_raw = row[4]
                unit_multiplier = int(row[5])

                # Execute the insert query
                cursor.execute(insert_query, (
                    year,
                    line_item,
                    column_name,
                    amount_raw,
                    unit_raw,
                    unit_multiplier
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{csv_file_path}' 파일의 데이터가 '환율변동위험' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")
except Exception as e:
    print(f"❌ 오류: 데이터 삽입 중 문제가 발생했습니다. 변경사항을 롤백합니다.")
    print(f"오류 내용: {e}")
    conn.rollback()
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

테이블 '환율변동위험'가 성공적으로 생성되었거나 이미 존재합니다.
✅ 'DB_csv/[환율변동위험]matrix_long_raw.csv' 파일의 데이터가 '환율변동위험' 테이블에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [60]:
import sqlite3
import csv
import os

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/[부채비율]matrix_long_raw.csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

try:
    # Create a new table for the debt ratio data
    create_table_query = """
    CREATE TABLE IF NOT EXISTS 부채비율 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        연도 INTEGER NOT NULL,
        항목 TEXT,
        변동내역 TEXT,
        금액 REAL,
        단위 TEXT,
        단위_배수 INTEGER
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 '부채비율'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO 부채비율 (
            연도,
            항목,
            변동내역,
            금액,
            단위,
            단위_배수
        ) VALUES (?, ?, ?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                year = int(row[0])
                line_item = row[1]
                column_name = row[2]
                amount_raw = float(row[3])
                unit_raw = row[4]
                unit_multiplier = int(row[5])

                # Execute the insert query
                cursor.execute(insert_query, (
                    year,
                    line_item,
                    column_name,
                    amount_raw,
                    unit_raw,
                    unit_multiplier
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{os.path.basename(csv_file_path)}' 파일의 데이터가 '부채비율' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")
except Exception as e:
    print(f"❌ 오류: 데이터 삽입 중 문제가 발생했습니다. 변경사항을 롤백합니다.")
    print(f"오류 내용: {e}")
    conn.rollback()
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

테이블 '부채비율'가 성공적으로 생성되었거나 이미 존재합니다.
✅ '[부채비율]matrix_long_raw.csv' 파일의 데이터가 '부채비율' 테이블에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [61]:
import sqlite3
import csv
import os

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/[공정가치]matrix_long_raw.csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

try:
    # Create a new table for the fair value data
    create_table_query = """
    CREATE TABLE IF NOT EXISTS 공정가치 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        연도 INTEGER NOT NULL,
        항목 TEXT,
        변동내역 TEXT,
        금액 REAL,
        단위 TEXT,
        단위_배수 INTEGER
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 '공정가치'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO 공정가치 (
            연도,
            항목,
            변동내역,
            금액,
            단위,
            단위_배수
        ) VALUES (?, ?, ?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                year = int(row[0])
                line_item = row[1]
                column_name = row[2]
                amount_raw = float(row[3])
                unit_raw = row[4]
                unit_multiplier = int(row[5])

                # Execute the insert query
                cursor.execute(insert_query, (
                    year,
                    line_item,
                    column_name,
                    amount_raw,
                    unit_raw,
                    unit_multiplier
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{os.path.basename(csv_file_path)}' 파일의 데이터가 '공정가치' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")
except Exception as e:
    print(f"❌ 오류: 데이터 삽입 중 문제가 발생했습니다. 변경사항을 롤백합니다.")
    print(f"오류 내용: {e}")
    conn.rollback()
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

테이블 '공정가치'가 성공적으로 생성되었거나 이미 존재합니다.
✅ '[공정가치]matrix_long_raw.csv' 파일의 데이터가 '공정가치' 테이블에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [62]:
import sqlite3
import csv
import os

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/[공정가치 측정치의 민감도분석]matrix_long_raw.csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

try:
    # Create a new table for the fair value measurement sensitivity analysis data with REAL for unit_multiplier
    create_table_query = """
    CREATE TABLE IF NOT EXISTS 공정가치_민감도분석 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        연도 INTEGER NOT NULL,
        항목 TEXT,
        변동내역 TEXT,
        금액 REAL,
        단위 TEXT,
        단위_배수 REAL
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 '공정가치_민감도분석'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO 공정가치_민감도분석 (
            연도,
            항목,
            변동내역,
            금액,
            단위,
            단위_배수
        ) VALUES (?, ?, ?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                year = int(row[0])
                line_item = row[1]
                column_name = row[2]
                amount_raw = float(row[3]) if row[3] else None
                unit_raw = row[4] if row[4] else None
                # Convert to float to handle '1.0'
                unit_multiplier = float(row[5]) if row[5] else None

                # Execute the insert query
                cursor.execute(insert_query, (
                    year,
                    line_item,
                    column_name,
                    amount_raw,
                    unit_raw,
                    unit_multiplier
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{os.path.basename(csv_file_path)}' 파일의 데이터가 '공정가치_민감도분석' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")
except Exception as e:
    print(f"❌ 오류: 데이터 삽입 중 문제가 발생했습니다. 변경사항을 롤백합니다.")
    print(f"오류 내용: {e}")
    conn.rollback()
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

테이블 '공정가치_민감도분석'가 성공적으로 생성되었거나 이미 존재합니다.
✅ '[공정가치 측정치의 민감도분석]matrix_long_raw.csv' 파일의 데이터가 '공정가치_민감도분석' 테이블에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [63]:
import sqlite3
import csv
import os

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/[부문별 보고]matrix_long_raw.csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

try:
    # Create a new table for the segmental reporting data
    create_table_query = """
    CREATE TABLE IF NOT EXISTS 부문별_보고 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        연도 INTEGER NOT NULL,
        항목 TEXT,
        변동내역 TEXT,
        금액 REAL,
        단위 TEXT,
        단위_배수 INTEGER
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 '부문별_보고'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO 부문별_보고 (
            연도,
            항목,
            변동내역,
            금액,
            단위,
            단위_배수
        ) VALUES (?, ?, ?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                year = int(row[0])
                line_item = row[1]
                column_name = row[2]
                amount_raw = float(row[3])
                unit_raw = row[4]
                unit_multiplier = int(row[5])

                # Execute the insert query
                cursor.execute(insert_query, (
                    year,
                    line_item,
                    column_name,
                    amount_raw,
                    unit_raw,
                    unit_multiplier
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{os.path.basename(csv_file_path)}' 파일의 데이터가 '부문별_보고' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")
except Exception as e:
    print(f"❌ 오류: 데이터 삽입 중 문제가 발생했습니다. 변경사항을 롤백합니다.")
    print(f"오류 내용: {e}")
    conn.rollback()
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

테이블 '부문별_보고'가 성공적으로 생성되었거나 이미 존재합니다.
✅ '[부문별 보고]matrix_long_raw.csv' 파일의 데이터가 '부문별_보고' 테이블에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [22]:
import sqlite3
import csv
import os

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/[Level3 금융상품의 변동내역]_merged.csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

try:
    # Create a new table for the Level 3 financial instruments data
    create_table_query = """
    CREATE TABLE IF NOT EXISTS Level3_금융상품_변동내역 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        연도 INTEGER NOT NULL,
        단위 TEXT,
        구분 TEXT,
        값 REAL
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 'Level3_금융상품_변동내역'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO Level3_금융상품_변동내역 (
            연도,
            단위,
            구분,
            값
        ) VALUES (?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                year = int(row[0])
                unit = row[1]
                category = row[2]
                value = float(row[3])

                # Execute the insert query
                cursor.execute(insert_query, (
                    year,
                    unit,
                    category,
                    value
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{os.path.basename(csv_file_path)}' 파일의 데이터가 'Level3_금융상품_변동내역' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")
except Exception as e:
    print(f"❌ 오류: 데이터 삽입 중 문제가 발생했습니다. 변경사항을 롤백합니다.")
    print(f"오류 내용: {e}")
    conn.rollback()
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

테이블 'Level3_금융상품_변동내역'가 성공적으로 생성되었거나 이미 존재합니다.
✅ '[Level3 금융상품의 변동내역]_merged.csv' 파일의 데이터가 'Level3_금융상품_변동내역' 테이블에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [64]:
import sqlite3
import csv
import os

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/[유동성위험]_merged.csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

try:
    # Create a new table for the liquidity risk data
    create_table_query = """
    CREATE TABLE IF NOT EXISTS 유동성위험 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        연도 INTEGER NOT NULL,
        단위 TEXT,
        구분 TEXT,
        기간 TEXT,
        값 REAL
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 '유동성위험'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO 유동성위험 (
            연도,
            단위,
            구분,
            기간,
            값
        ) VALUES (?, ?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                year = int(row[0])
                unit = row[1]
                category = row[2]
                period = row[3]
                value = float(row[4])

                # Execute the insert query
                cursor.execute(insert_query, (
                    year,
                    unit,
                    category,
                    period,
                    value
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{os.path.basename(csv_file_path)}' 파일의 데이터가 '유동성위험' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")
except Exception as e:
    print(f"❌ 오류: 데이터 삽입 중 문제가 발생했습니다. 변경사항을 롤백합니다.")
    print(f"오류 내용: {e}")
    conn.rollback()
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

테이블 '유동성위험'가 성공적으로 생성되었거나 이미 존재합니다.
✅ '[유동성위험]_merged.csv' 파일의 데이터가 '유동성위험' 테이블에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [65]:
import sqlite3
import csv
import os

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/[공정가치 수준별 공시]_merged .csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

try:
    # Create a new table for the fair value level data
    create_table_query = """
    CREATE TABLE IF NOT EXISTS 공정가치_수준별 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        연도 INTEGER NOT NULL,
        자산부채 TEXT,
        구분 TEXT,
        수준 INTEGER,
        값 REAL,
        단위 TEXT
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 '공정가치_수준별'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO 공정가치_수준별 (
            연도,
            자산부채,
            구분,
            수준,
            값,
            단위
        ) VALUES (?, ?, ?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                year = int(row[0])
                asset_liability = row[1]
                category = row[2]
                level = int(row[3])
                value = float(row[4])
                unit = row[5]

                # Execute the insert query
                cursor.execute(insert_query, (
                    year,
                    asset_liability,
                    category,
                    level,
                    value,
                    unit
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{os.path.basename(csv_file_path)}' 파일의 데이터가 '공정가치_수준별' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")
except Exception as e:
    print(f"❌ 오류: 데이터 삽입 중 문제가 발생했습니다. 변경사항을 롤백합니다.")
    print(f"오류 내용: {e}")
    conn.rollback()
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

테이블 '공정가치_수준별'가 성공적으로 생성되었거나 이미 존재합니다.
✅ '[공정가치 수준별 공시]_merged .csv' 파일의 데이터가 '공정가치_수준별' 테이블에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [66]:
import sqlite3

# 확인하고 싶은 테이블 목록
tables = [
    '환율변동위험',
    '부채비율',
    '공정가치',
    '공정가치_민감도분석',
    '부문별_보고',
    'Level3_금융상품_변동내역',
    '유동성위험',
    '공정가치_수준별'
]

# 데이터베이스에 연결
conn = sqlite3.connect('samsung_audit_report.db')
cursor = conn.cursor()

try:
    print("=== 데이터베이스 테이블 확인 ===")
    for table_name in tables:
        print(f"\n--- 테이블: {table_name} ---")
        
        # 총 행의 개수 확인
        count_query = f"SELECT COUNT(*) FROM {table_name};"
        cursor.execute(count_query)
        count = cursor.fetchone()[0]
        print(f"총 데이터 개수: {count}개")
        
        # 상위 5개 행 조회
        select_query = f"SELECT * FROM {table_name} LIMIT 5;"
        cursor.execute(select_query)
        rows = cursor.fetchall()

        if rows:
            # 컬럼 이름 가져오기
            columns = [description[0] for description in cursor.description]
            print(f"컬럼: {', '.join(columns)}")
            
            # 데이터 출력
            for row in rows:
                print(row)
        else:
            print("데이터가 없습니다.")

except sqlite3.OperationalError as e:
    print(f"오류: {e}")
    print("테이블 이름이 올바르지 않거나, 테이블이 존재하지 않습니다.")

finally:
    # 연결 종료
    cursor.close()
    conn.close()
    print("\n=== 확인 완료 ===")

=== 데이터베이스 테이블 확인 ===

--- 테이블: 환율변동위험 ---
총 데이터 개수: 66개
컬럼: id, 연도, 항목, 변동내역, 금액, 단위, 단위_배수
(1, 2014, 'USD', '당기말 / 환율 상승시', 134051.0, '백만원', 1)
(2, 2014, 'USD', '당기말 / 환율 하락시', -134051.0, '백만원', 1)
(3, 2014, 'EUR', '당기말 / 환율 상승시', 53007.0, '백만원', 1)
(4, 2014, 'EUR', '당기말 / 환율 하락시', -53007.0, '백만원', 1)
(5, 2014, 'JPY', '당기말 / 환율 상승시', -1254.0, '백만원', 1)

--- 테이블: 부채비율 ---
총 데이터 개수: 30개
컬럼: id, 연도, 항목, 변동내역, 금액, 단위, 단위_배수
(1, 2014, '부 \xa0채', '당기말', 31383508.0, '백만원', 1)
(2, 2014, '자 \xa0본', '당기말', 132677075.0, '백만원', 1)
(3, 2014, '부채비율', '당기말', 23.7, '백만원', 1)
(4, 2015, '부 \xa0채', '당기말', 32541375.0, '백만원', 1)
(5, 2015, '자 \xa0본', '당기말', 136428255.0, '백만원', 1)

--- 테이블: 공정가치 ---
총 데이터 개수: 200개
컬럼: id, 연도, 항목, 변동내역, 금액, 단위, 단위_배수
(1, 2014, '금융자산 / 현금및현금성자산', '당기말 / 장부금액', 1643318.0, '백만원', 1)
(2, 2014, '금융자산 / 단기금융상품', '당기말 / 장부금액', 26454093.0, '백만원', 1)
(3, 2014, '금융자산 / 단기매도가능금융자산', '당기말 / 장부금액', 2137533.0, '백만원', 1)
(4, 2014, '금융자산 / 단기매도가능금융자산', '당기말 / 공정가치', 2137533.0, '백만원', 1)
(5

# 태완 CSV

In [72]:
import sqlite3
import csv
import os

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/거래내역_all.csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

def convert_to_float_or_none(value):
    """
    Converts a string to a float, handling hyphens and empty strings as None.
    """
    if value is None or value.strip() == '-' or value.strip() == '':
        return None
    try:
        return float(value.replace(',', ''))
    except (ValueError, IndexError):
        return None

try:
    # Create a new table for the transaction data
    create_table_query = """
    CREATE TABLE IF NOT EXISTS 거래내역 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        구분 TEXT,
        기업명 TEXT,
        매출_등 REAL,
        비유동자산_처분 REAL,
        매입_등 REAL,
        비유동자산_매입 REAL,
        단위 TEXT,
        연도 INTEGER NOT NULL
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 '거래내역'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO 거래내역 (
            구분,
            기업명,
            매출_등,
            비유동자산_처분,
            매입_등,
            비유동자산_매입,
            단위,
            연도
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                category = row[0]
                company_name = row[1]
                sales = convert_to_float_or_none(row[2])
                non_current_asset_disposal = convert_to_float_or_none(row[3])
                purchases = convert_to_float_or_none(row[4])
                non_current_asset_purchase = convert_to_float_or_none(row[5])
                unit = row[6]
                year = int(row[7])

                # Execute the insert query
                cursor.execute(insert_query, (
                    category,
                    company_name,
                    sales,
                    non_current_asset_disposal,
                    purchases,
                    non_current_asset_purchase,
                    unit,
                    year
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{os.path.basename(csv_file_path)}' 파일의 데이터가 '거래내역' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")
except Exception as e:
    print(f"❌ 오류: 데이터 삽입 중 문제가 발생했습니다. 변경사항을 롤백합니다.")
    print(f"오류 내용: {e}")
    conn.rollback()
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

테이블 '거래내역'가 성공적으로 생성되었거나 이미 존재합니다.
✅ '거래내역_all.csv' 파일의 데이터가 '거래내역' 테이블에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [73]:
import sqlite3
import csv
import os

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/경영진보상_all.csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

def convert_to_float_or_none(value):
    """
    Converts a string to a float, handling commas and empty strings as None.
    """
    if value is None or value.strip() == '':
        return None
    try:
        return float(value.replace(',', ''))
    except (ValueError, IndexError):
        return None

try:
    # Create a new table for the executive compensation data
    create_table_query = """
    CREATE TABLE IF NOT EXISTS 경영진보상 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        구분 TEXT,
        당기 REAL,
        전기 REAL,
        단위 TEXT,
        연도 INTEGER NOT NULL
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 '경영진보상'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO 경영진보상 (
            구분,
            당기,
            전기,
            단위,
            연도
        ) VALUES (?, ?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                category = row[0]
                current_period = convert_to_float_or_none(row[1])
                previous_period = convert_to_float_or_none(row[2])
                unit = row[3]
                year = int(row[4])

                # Execute the insert query
                cursor.execute(insert_query, (
                    category,
                    current_period,
                    previous_period,
                    unit,
                    year
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{os.path.basename(csv_file_path)}' 파일의 데이터가 '경영진보상' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")
except Exception as e:
    print(f"❌ 오류: 데이터 삽입 중 문제가 발생했습니다. 변경사항을 롤백합니다.")
    print(f"오류 내용: {e}")
    conn.rollback()
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

테이블 '경영진보상'가 성공적으로 생성되었거나 이미 존재합니다.
✅ '경영진보상_all.csv' 파일의 데이터가 '경영진보상' 테이블에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [74]:
import sqlite3
import csv
import os

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/지배관계_all.csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

def clean_and_convert_stake(value):
    """
    Cleans a string by removing special characters and converts it to a float.
    Returns None if conversion fails.
    """
    if not value or value.strip() == '':
        return None
    try:
        # Remove '(*)' and any other non-numeric characters before conversion
        cleaned_value = value.replace('(*)', '').strip()
        return float(cleaned_value)
    except (ValueError, IndexError):
        return None

try:
    # Create a new table for the governance data
    create_table_query = """
    CREATE TABLE IF NOT EXISTS 지배관계 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        지역 TEXT,
        기업명 TEXT,
        업종 TEXT,
        지분율_퍼센트 REAL,
        연도 INTEGER NOT NULL
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 '지배관계'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO 지배관계 (
            지역,
            기업명,
            업종,
            지분율_퍼센트,
            연도
        ) VALUES (?, ?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                region = row[0]
                company_name = row[1]
                industry = row[2]
                stake_percent = clean_and_convert_stake(row[3])
                year = int(row[4])

                # Execute the insert query
                cursor.execute(insert_query, (
                    region,
                    company_name,
                    industry,
                    stake_percent,
                    year
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{os.path.basename(csv_file_path)}' 파일의 데이터가 '지배관계' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")
except Exception as e:
    print(f"❌ 오류: 데이터 삽입 중 문제가 발생했습니다. 변경사항을 롤백합니다.")
    print(f"오류 내용: {e}")
    conn.rollback()
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

테이블 '지배관계'가 성공적으로 생성되었거나 이미 존재합니다.
✅ '지배관계_all.csv' 파일의 데이터가 '지배관계' 테이블에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [75]:
import sqlite3
import csv
import os

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/채권채무_all.csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

def convert_to_float_or_none(value):
    """
    Converts a string to a float, handling commas, hyphens and empty strings as None.
    """
    if value is None or value.strip() == '-' or value.strip() == '':
        return None
    try:
        return float(value.replace(',', ''))
    except (ValueError, IndexError):
        return None

try:
    # Create a new table for the receivables and payables data
    create_table_query = """
    CREATE TABLE IF NOT EXISTS 채권채무 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        구분 TEXT,
        기업명 TEXT,
        채권_등 REAL,
        채무_등 REAL,
        단위 TEXT,
        연도 INTEGER NOT NULL
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 '채권채무'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO 채권채무 (
            구분,
            기업명,
            채권_등,
            채무_등,
            단위,
            연도
        ) VALUES (?, ?, ?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                category = row[0]
                company_name = row[1]
                receivables = convert_to_float_or_none(row[2])
                payables = convert_to_float_or_none(row[3])
                unit = row[4]
                year = int(row[5])

                # Execute the insert query
                cursor.execute(insert_query, (
                    category,
                    company_name,
                    receivables,
                    payables,
                    unit,
                    year
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{os.path.basename(csv_file_path)}' 파일의 데이터가 '채권채무' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")
except Exception as e:
    print(f"❌ 오류: 데이터 삽입 중 문제가 발생했습니다. 변경사항을 롤백합니다.")
    print(f"오류 내용: {e}")
    conn.rollback()
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

테이블 '채권채무'가 성공적으로 생성되었거나 이미 존재합니다.
✅ '채권채무_all.csv' 파일의 데이터가 '채권채무' 테이블에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [ ]:
import sqlite3

# 확인하고 싶은 테이블 목록
tables = [
    '거래내역',
    '경영진보상',
    '지배관계',
    '채권채무',
]

# 데이터베이스에 연결
conn = sqlite3.connect('samsung_audit_report.db')
cursor = conn.cursor()

try:
    print("=== 데이터베이스 테이블 확인 ===")
    for table_name in tables:
        print(f"\n--- 테이블: {table_name} ---")+
        
        # 총 행의 개수 확인
        count_query = f"SELECT COUNT(*) FROM {table_name};"
        cursor.execute(count_query)
        count = cursor.fetchone()[0]
        print(f"총 데이터 개수: {count}개")
        
        # 상위 5개 행 조회
        select_query = f"SELECT * FROM {table_name} LIMIT 5;"
        cursor.execute(select_query)
        rows = cursor.fetchall()

        if rows:
            # 컬럼 이름 가져오기
            columns = [description[0] for description in cursor.description]
            print(f"컬럼: {', '.join(columns)}")
            
            # 데이터 출력
            for row in rows:
                print(row)
        else:
            print("데이터가 없습니다.")

except sqlite3.OperationalError as e:
    print(f"오류: {e}")
    print("테이블 이름이 올바르지 않거나, 테이블이 존재하지 않습니다.")

finally:
    # 연결 종료
    cursor.close()
    conn.close()
    print("\n=== 확인 완료 ===")

=== 데이터베이스 테이블 확인 ===

--- 테이블: 거래내역 ---
총 데이터 개수: 434개
컬럼: id, 구분, 기업명, 매출_등, 비유동자산_처분, 매입_등, 비유동자산_매입, 단위, 연도
(1, '종속기업', '삼성디스플레이', 802934.0, None, 2666926.0, 1463.0, '백만원', 2014)
(2, '종속기업', 'Samsung Electronics America(SEA)', 12625057.0, None, 8170.0, None, '백만원', 2014)
(3, '종속기업', 'Samsung (China) Investment(SCIC)', 755971.0, None, 9741.0, None, '백만원', 2014)
(4, '종속기업', 'Samsung Semiconductor(SSI)', 13432528.0, None, 235801.0, 407.0, '백만원', 2014)
(5, '종속기업', 'Samsung Electronics Europe Holding(SEEH)', 1722.0, None, None, None, '백만원', 2014)

--- 테이블: 경영진보상 ---
총 데이터 개수: 33개
컬럼: id, 구분, 당기, 전기, 단위, 연도
(1, '단기급여', 26969.0, 23906.0, '백만원', 2014)
(2, '퇴직급여', 973.0, 763.0, '백만원', 2014)
(3, '기타 장기급여', 7137.0, 7402.0, '백만원', 2014)
(4, '단기급여', 23671.0, 26969.0, '백만원', 2015)
(5, '퇴직급여', 560.0, 973.0, '백만원', 2015)

--- 테이블: 지배관계 ---
총 데이터 개수: 2409개
컬럼: id, 지역, 기업명, 업종, 지분율_퍼센트, 연도
(1, '국내', '하이파이오니어 사모특별자산투자신탁1호', '신기술사업자, 벤처기업 투자', 100.0, 2014)
(2, '국내', '삼성디스플레이', '디스플레이 제조 및 판매', 84.8, 2

# 기태 CSV

In [41]:
import sqlite3
import csv
import os

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/[기타자본]matrix_long_processed.csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

def convert_to_float_or_none(value):
    """
    Converts a string to a float, handling empty strings as None.
    """
    if value is None or value.strip() == '':
        return None
    try:
        return float(value.replace(',', ''))
    except (ValueError, IndexError):
        return None

try:
    # Create a new table for the other capital data
    create_table_query = """
    CREATE TABLE IF NOT EXISTS 기타자본 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        연도 INTEGER NOT NULL,
        항목 TEXT,
        금액 REAL,
        단위 TEXT
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 '기타자본'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO 기타자본 (
            연도,
            항목,
            금액,
            단위
        ) VALUES (?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                year = int(row[0])
                line_item = row[1]
                amount = convert_to_float_or_none(row[2])
                unit = row[3]

                # Execute the insert query
                cursor.execute(insert_query, (
                    year,
                    line_item,
                    amount,
                    unit
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{os.path.basename(csv_file_path)}' 파일의 데이터가 '기타자본' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")
except Exception as e:
    print(f"❌ 오류: 데이터 삽입 중 문제가 발생했습니다. 변경사항을 롤백합니다.")
    print(f"오류 내용: {e}")
    conn.rollback()
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

테이블 '기타자본'가 성공적으로 생성되었거나 이미 존재합니다.
✅ '[기타자본]matrix_long_processed.csv' 파일의 데이터가 '기타자본' 테이블에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [44]:
import sqlite3
import csv
import os

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/[법인세]matrix_long_processed.csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

def convert_to_float_or_none(value):
    """
    Converts a string to a float, handling commas and empty strings as None.
    """
    if value is None or value.strip() == '':
        return None
    try:
        return float(value.replace(',', ''))
    except (ValueError, IndexError):
        return None

try:
    # Create a new table for the corporate tax data
    create_table_query = """
    CREATE TABLE IF NOT EXISTS 법인세 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        연도 INTEGER NOT NULL,
        항목 TEXT,
        금액 REAL,
        단위 TEXT
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 '법인세'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO 법인세 (
            연도,
            항목,
            금액,
            단위
        ) VALUES (?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                year = int(row[0])
                line_item = row[1]
                amount = convert_to_float_or_none(row[2])
                unit = row[3]

                # Execute the insert query
                cursor.execute(insert_query, (
                    year,
                    line_item,
                    amount,
                    unit
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{os.path.basename(csv_file_path)}' 파일의 데이터가 '법인세' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")
except Exception as e:
    print(f"❌ 오류: 데이터 삽입 중 문제가 발생했습니다. 변경사항을 롤백합니다.")
    print(f"오류 내용: {e}")
    conn.rollback()
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

테이블 '법인세'가 성공적으로 생성되었거나 이미 존재합니다.
✅ '[법인세]matrix_long_processed.csv' 파일의 데이터가 '법인세' 테이블에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [45]:
import sqlite3
import csv
import os

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/[비용수익]matrix_long_processed.csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

def convert_to_float_or_none(value):
    """
    Converts a string to a float, handling commas and empty strings as None.
    """
    if value is None or value.strip() == '':
        return None
    try:
        return float(value.replace(',', ''))
    except (ValueError, IndexError):
        return None

try:
    # Create a new table for the expenses and revenue data
    create_table_query = """
    CREATE TABLE IF NOT EXISTS 비용수익 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        연도 INTEGER NOT NULL,
        항목 TEXT,
        금액 REAL,
        단위 TEXT
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 '비용수익'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO 비용수익 (
            연도,
            항목,
            금액,
            단위
        ) VALUES (?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                year = int(row[0])
                line_item = row[1]
                amount = convert_to_float_or_none(row[2])
                unit = row[3]

                # Execute the insert query
                cursor.execute(insert_query, (
                    year,
                    line_item,
                    amount,
                    unit
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{os.path.basename(csv_file_path)}' 파일의 데이터가 '비용수익' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")
except Exception as e:
    print(f"❌ 오류: 데이터 삽입 중 문제가 발생했습니다. 변경사항을 롤백합니다.")
    print(f"오류 내용: {e}")
    conn.rollback()
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

테이블 '비용수익'가 성공적으로 생성되었거나 이미 존재합니다.
✅ '[비용수익]matrix_long_processed.csv' 파일의 데이터가 '비용수익' 테이블에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [47]:
import sqlite3
import csv
import os

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/[자본금_이익잉여금]matrix_long_processed.csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

def convert_to_float_or_none(value):
    """
    Converts a string to a float, handling commas and empty strings as None.
    """
    if value is None or value.strip() == '':
        return None
    try:
        return float(value.replace(',', ''))
    except (ValueError, IndexError):
        return None

try:
    # Create a new table for the capital stock and retained earnings data
    create_table_query = """
    CREATE TABLE IF NOT EXISTS 자본금_이익잉여금 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        연도 INTEGER NOT NULL,
        항목 TEXT,
        금액 REAL,
        단위 TEXT
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 '자본금_이익잉여금'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO 자본금_이익잉여금 (
            연도,
            항목,
            금액,
            단위
        ) VALUES (?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                year = int(row[0])
                line_item = row[1]
                amount = convert_to_float_or_none(row[2])
                unit = row[3]

                # Execute the insert query
                cursor.execute(insert_query, (
                    year,
                    line_item,
                    amount,
                    unit
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{os.path.basename(csv_file_path)}' 파일의 데이터가 '자본금_이익잉여금' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")
except Exception as e:
    print(f"❌ 오류: 데이터 삽입 중 문제가 발생했습니다. 변경사항을 롤백합니다.")
    print(f"오류 내용: {e}")
    conn.rollback()
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

테이블 '자본금_이익잉여금'가 성공적으로 생성되었거나 이미 존재합니다.
✅ '[자본금_이익잉여금]matrix_long_processed.csv' 파일의 데이터가 '자본금_이익잉여금' 테이블에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [48]:
import sqlite3
import csv
import os

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/[주당이익_배당]matrix_long_processed.csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

def convert_to_float_or_none(value):
    """
    Converts a string to a float, handling commas and empty strings as None.
    """
    if value is None or value.strip() == '':
        return None
    try:
        return float(value.replace(',', ''))
    except (ValueError, IndexError):
        return None

try:
    # Create a new table for the EPS and dividends data
    create_table_query = """
    CREATE TABLE IF NOT EXISTS 주당이익_배당 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        연도 INTEGER NOT NULL,
        항목 TEXT,
        금액 REAL,
        단위 TEXT
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 '주당이익_배당'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO 주당이익_배당 (
            연도,
            항목,
            금액,
            단위
        ) VALUES (?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                year = int(row[0])
                line_item = row[1]
                amount = convert_to_float_or_none(row[2])
                unit = row[3]

                # Execute the insert query
                cursor.execute(insert_query, (
                    year,
                    line_item,
                    amount,
                    unit
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{os.path.basename(csv_file_path)}' 파일의 데이터가 '주당이익_배당' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")
except Exception as e:
    print(f"❌ 오류: 데이터 삽입 중 문제가 발생했습니다. 변경사항을 롤백합니다.")
    print(f"오류 내용: {e}")
    conn.rollback()
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

테이블 '주당이익_배당'가 성공적으로 생성되었거나 이미 존재합니다.
✅ '[주당이익_배당]matrix_long_processed.csv' 파일의 데이터가 '주당이익_배당' 테이블에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [49]:
import sqlite3

# 확인하고 싶은 테이블 목록
tables = [
    '기타자본',
    '법인세',
    '비용수익',
    '자본금_이익잉여금',
    '주당이익_배당',
]

# 데이터베이스에 연결
conn = sqlite3.connect('samsung_audit_report.db')
cursor = conn.cursor()

try:
    print("=== 데이터베이스 테이블 확인 ===")
    for table_name in tables:
        print(f"\n--- 테이블: {table_name} ---")
        
        # 총 행의 개수 확인
        count_query = f"SELECT COUNT(*) FROM {table_name};"
        cursor.execute(count_query)
        count = cursor.fetchone()[0]
        print(f"총 데이터 개수: {count}개")
        
        # 상위 5개 행 조회
        select_query = f"SELECT * FROM {table_name} LIMIT 5;"
        cursor.execute(select_query)
        rows = cursor.fetchall()

        if rows:
            # 컬럼 이름 가져오기
            columns = [description[0] for description in cursor.description]
            print(f"컬럼: {', '.join(columns)}")
            
            # 데이터 출력
            for row in rows:
                print(row)
        else:
            print("데이터가 없습니다.")

except sqlite3.OperationalError as e:
    print(f"오류: {e}")
    print("테이블 이름이 올바르지 않거나, 테이블이 존재하지 않습니다.")

finally:
    # 연결 종료
    cursor.close()
    conn.close()
    print("\n=== 확인 완료 ===")

=== 데이터베이스 테이블 확인 ===

--- 테이블: 기타자본 ---
총 데이터 개수: 28개
컬럼: id, 연도, 항목, 금액, 단위
(1, 2014, '자기주식', -8429313.0, '백만원')
(2, 2014, '주식선택권', 806.0, '백만원')
(3, 2014, '매도가능금융자산평가손익', 1222820.0, '백만원')
(4, 2014, '순확정급여부채 재측정요소', -1666632.0, '백만원')
(5, 2014, '기타', 1759048.0, '백만원')

--- 테이블: 법인세 ---
총 데이터 개수: 1256개
컬럼: id, 연도, 항목, 금액, 단위
(1, 2014, '기간손익에 대한 당기법인세', 3940377.0, '백만원')
(2, 2014, '당기에 인식한 조정사항', 14830.0, '백만원')
(3, 2014, '2. 이연법인세', None, '백만원')
(4, 2014, '세액공제의 이월로 인한 이연법인세변동액', 634775.0, '백만원')
(5, 2014, '일시적차이로 인한 이연법인세변동액', -1896868.0, '백만원')

--- 테이블: 비용수익 ---
총 데이터 개수: 478개
컬럼: id, 연도, 항목, 금액, 단위
(1, 2014, '매출원가', 339985.0, '백만원')
(2, 2014, '판매비와관리비 등', 569933.0, '백만원')
(3, 2014, '급여', 1744060.0, '백만원')
(4, 2014, '퇴직급여', 186627.0, '백만원')
(5, 2014, '지급수수료', 3973538.0, '백만원')

--- 테이블: 자본금_이익잉여금 ---
총 데이터 개수: 81개
컬럼: id, 연도, 항목, 금액, 단위
(1, 2014, '법정적립금', 450789.0, '백만원')
(2, 2014, '임의적립금 : 재무구조개선적립금', 204815.0, '백만원')
(3, 2014, '임의적립금 : 기업합리화적립금', 26512101.0, '백만원')
(4, 2014, '임의

# 희선 CSV

In [77]:
import sqlite3
import csv
import os

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/매각예정분류_통합테이블_2014-2023.csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

def convert_to_float_or_none(value):
    """
    Converts a string to a float, handling commas and empty strings as None.
    """
    if value is None or value.strip() == '':
        return None
    try:
        return float(value.replace(',', ''))
    except (ValueError, IndexError):
        return None

try:
    # Create a new table for the assets classified as held-for-sale data
    create_table_query = """
    CREATE TABLE IF NOT EXISTS 매각예정분류 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        연도 INTEGER NOT NULL,
        항목 TEXT,
        변동내역 TEXT,
        금액 REAL,
        단위 TEXT,
        값_유형 TEXT
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 '매각예정분류'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO 매각예정분류 (
            연도,
            항목,
            변동내역,
            금액,
            단위,
            값_유형
        ) VALUES (?, ?, ?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                year = int(row[0])
                line_item = row[1]
                column_name = row[2]
                amount = convert_to_float_or_none(row[3])
                unit = row[4]
                value_type = row[5]

                # Execute the insert query
                cursor.execute(insert_query, (
                    year,
                    line_item,
                    column_name,
                    amount,
                    unit,
                    value_type
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{os.path.basename(csv_file_path)}' 파일의 데이터가 '매각예정분류' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")
except Exception as e:
    print(f"❌ 오류: 데이터 삽입 중 문제가 발생했습니다. 변경사항을 롤백합니다.")
    print(f"오류 내용: {e}")
    conn.rollback()
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

테이블 '매각예정분류'가 성공적으로 생성되었거나 이미 존재합니다.
✅ '매각예정분류_통합테이블_2014-2023.csv' 파일의 데이터가 '매각예정분류' 테이블에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [78]:
import sqlite3
import csv
import os

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/무상할당 배출권 및 온실가스 매출량 추정치_통합테이블_2014-2023.csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

def convert_to_float_or_none(value):
    """
    Converts a string to a float, handling commas, quotes, and empty strings as None.
    """
    if value is None or value.strip() == '':
        return None
    try:
        # Remove commas and quotes before conversion
        cleaned_value = value.replace(',', '').replace('"', '').strip()
        return float(cleaned_value)
    except (ValueError, IndexError):
        return None

try:
    # Create a new table for the emission allowance data
    create_table_query = """
    CREATE TABLE IF NOT EXISTS 무상할당_배출권 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        연도 INTEGER NOT NULL,
        변동내역 TEXT,
        단위 TEXT,
        무상할당_배출권 REAL,
        배출량_추정치 REAL
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 '무상할당_배출권'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO 무상할당_배출권 (
            연도,
            변동내역,
            단위,
            무상할당_배출권,
            배출량_추정치
        ) VALUES (?, ?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                year = int(row[0])
                column_name = row[1]
                unit = row[2]
                allowance = convert_to_float_or_none(row[3])
                emissions = convert_to_float_or_none(row[4])

                # Execute the insert query
                cursor.execute(insert_query, (
                    year,
                    column_name,
                    unit,
                    allowance,
                    emissions
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{os.path.basename(csv_file_path)}' 파일의 데이터가 '무상할당_배출권' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")
except Exception as e:
    print(f"❌ 오류: 데이터 삽입 중 문제가 발생했습니다. 변경사항을 롤백합니다.")
    print(f"오류 내용: {e}")
    conn.rollback()
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

테이블 '무상할당_배출권'가 성공적으로 생성되었거나 이미 존재합니다.
✅ '무상할당 배출권 및 온실가스 매출량 추정치_통합테이블_2014-2023.csv' 파일의 데이터가 '무상할당_배출권' 테이블에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [79]:
import sqlite3
import csv
import os

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/사외적립자산_공정가치_변동내역.csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

def convert_to_float_or_none(value):
    """
    Converts a string to a float, handling commas, quotes, and empty strings as None.
    """
    if value is None or value.strip() == '':
        return None
    try:
        # Remove commas and quotes before conversion
        cleaned_value = value.replace(',', '').replace('"', '').strip()
        return float(cleaned_value)
    except (ValueError, IndexError):
        return None

try:
    # Create a new table for the external reserve assets data
    create_table_query = """
    CREATE TABLE IF NOT EXISTS 사외적립자산 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        연도 INTEGER NOT NULL,
        변동내역 TEXT,
        단위 TEXT,
        급여지급액 REAL,
        기말 REAL,
        기초 REAL,
        기타 REAL,
        이자수익 REAL,
        재측정요소 REAL
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 '사외적립자산'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO 사외적립자산 (
            연도,
            변동내역,
            단위,
            급여지급액,
            기말,
            기초,
            기타,
            이자수익,
            재측정요소
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                year = int(row[0])
                column_name = row[1]
                unit = row[2]
                salaries = convert_to_float_or_none(row[3])
                ending_balance = convert_to_float_or_none(row[4])
                beginning_balance = convert_to_float_or_none(row[5])
                other = convert_to_float_or_none(row[6])
                interest_income = convert_to_float_or_none(row[7])
                remeasurement = convert_to_float_or_none(row[8])

                # Execute the insert query
                cursor.execute(insert_query, (
                    year,
                    column_name,
                    unit,
                    salaries,
                    ending_balance,
                    beginning_balance,
                    other,
                    interest_income,
                    remeasurement
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{os.path.basename(csv_file_path)}' 파일의 데이터가 '사외적립자산' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")
except Exception as e:
    print(f"❌ 오류: 데이터 삽입 중 문제가 발생했습니다. 변경사항을 롤백합니다.")
    print(f"오류 내용: {e}")
    conn.rollback()
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

테이블 '사외적립자산'가 성공적으로 생성되었거나 이미 존재합니다.
✅ '사외적립자산_공정가치_변동내역.csv' 파일의 데이터가 '사외적립자산' 테이블에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [80]:
import sqlite3
import csv
import os

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/주요_가정의_변동에_따른_확정급여채무의_민감도_분석__통합테이블_2014-2023.csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

def convert_to_float_or_none(value):
    """
    Converts a string to a float, handling commas, colons, spaces, and empty strings as None.
    """
    if value is None or value.strip() == '':
        return None
    try:
        # Remove any non-numeric characters before conversion
        cleaned_value = value.replace(',', '').replace(':', '').strip()
        return float(cleaned_value)
    except (ValueError, IndexError):
        return None

try:
    # Create a new table for the defined benefit obligation sensitivity analysis data
    create_table_query = """
    CREATE TABLE IF NOT EXISTS 확정급여채무_민감도분석 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        연도 INTEGER NOT NULL,
        변동내역 TEXT,
        단위 TEXT,
        임금상승률_감소 REAL,
        임금상승률_증가 REAL,
        할인율_감소 REAL,
        할인율_증가 REAL
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 '확정급여채무_민감도분석'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO 확정급여채무_민감도분석 (
            연도,
            변동내역,
            단위,
            임금상승률_감소,
            임금상승률_증가,
            할인율_감소,
            할인율_증가
        ) VALUES (?, ?, ?, ?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                year = int(row[0])
                column_name = row[1]
                unit = row[2]
                wage_decrease = convert_to_float_or_none(row[3])
                wage_increase = convert_to_float_or_none(row[4])
                discount_rate_decrease = convert_to_float_or_none(row[5])
                discount_rate_increase = convert_to_float_or_none(row[6])

                # Execute the insert query
                cursor.execute(insert_query, (
                    year,
                    column_name,
                    unit,
                    wage_decrease,
                    wage_increase,
                    discount_rate_decrease,
                    discount_rate_increase
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{os.path.basename(csv_file_path)}' 파일의 데이터가 '확정급여채무_민감도분석' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")
except Exception as e:
    print(f"❌ 오류: 데이터 삽입 중 문제가 발생했습니다. 변경사항을 롤백합니다.")
    print(f"오류 내용: {e}")
    conn.rollback()
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

테이블 '확정급여채무_민감도분석'가 성공적으로 생성되었거나 이미 존재합니다.
✅ '주요_가정의_변동에_따른_확정급여채무의_민감도_분석__통합테이블_2014-2023.csv' 파일의 데이터가 '확정급여채무_민감도분석' 테이블에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [81]:
import sqlite3
import csv
import os

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/주요_보험수리적_가정_통합테이블_2014-2023.csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

def convert_to_float_or_none(value):
    """
    Converts a string to a float, handling commas, spaces, and empty strings as None.
    """
    if value is None or value.strip() == '':
        return None
    try:
        # Remove commas and spaces before conversion
        cleaned_value = value.replace(',', '').replace(' ', '').strip()
        return float(cleaned_value)
    except (ValueError, IndexError):
        return None

try:
    # Create a new table for the actuarial assumptions data
    create_table_query = """
    CREATE TABLE IF NOT EXISTS 보험수리적_가정 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        연도 INTEGER NOT NULL,
        변동내역 TEXT,
        단위 TEXT,
        미래임금상승률 REAL,
        할인율 REAL
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 '보험수리적_가정'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO 보험수리적_가정 (
            연도,
            변동내역,
            단위,
            미래임금상승률,
            할인율
        ) VALUES (?, ?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                year = int(row[0])
                column_name = row[1]
                unit = row[2]
                wage_growth = convert_to_float_or_none(row[3])
                discount_rate = convert_to_float_or_none(row[4])

                # Execute the insert query
                cursor.execute(insert_query, (
                    year,
                    column_name,
                    unit,
                    wage_growth,
                    discount_rate
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{os.path.basename(csv_file_path)}' 파일의 데이터가 '보험수리적_가정' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")
except Exception as e:
    print(f"❌ 오류: 데이터 삽입 중 문제가 발생했습니다. 변경사항을 롤백합니다.")
    print(f"오류 내용: {e}")
    conn.rollback()
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

테이블 '보험수리적_가정'가 성공적으로 생성되었거나 이미 존재합니다.
✅ '주요_보험수리적_가정_통합테이블_2014-2023.csv' 파일의 데이터가 '보험수리적_가정' 테이블에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [82]:
import sqlite3
import csv
import os

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/충당부채_통합테이블_2014-2023.csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

def convert_to_float_or_none(value):
    """
    Converts a string to a float, handling commas, quotes, parentheses, and empty strings as None.
    """
    if value is None or value.strip() == '':
        return None
    try:
        # Remove commas and quotes before conversion
        cleaned_value = value.replace(',', '').replace('"', '').strip()
        # Handle parentheses for negative numbers
        if cleaned_value.startswith('(') and cleaned_value.endswith(')'):
            cleaned_value = '-' + cleaned_value[1:-1]
        return float(cleaned_value)
    except (ValueError, IndexError):
        return None

try:
    # Create a new table for the provisions data
    create_table_query = """
    CREATE TABLE IF NOT EXISTS 충당부채 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        연도 INTEGER NOT NULL,
        항목 TEXT,
        판매보증 REAL,
        기술사용료 REAL,
        장기성과급 REAL,
        기타 REAL,
        계 REAL
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 '충당부채'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO 충당부채 (
            연도,
            항목,
            판매보증,
            기술사용료,
            장기성과급,
            기타,
            계
        ) VALUES (?, ?, ?, ?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                year = int(row[0])
                line_item = row[1]
                sales_warranty = convert_to_float_or_none(row[2])
                royalty = convert_to_float_or_none(row[3])
                long_term_incentive = convert_to_float_or_none(row[4])
                other = convert_to_float_or_none(row[5])
                total = convert_to_float_or_none(row[6])

                # Execute the insert query
                cursor.execute(insert_query, (
                    year,
                    line_item,
                    sales_warranty,
                    royalty,
                    long_term_incentive,
                    other,
                    total
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{os.path.basename(csv_file_path)}' 파일의 데이터가 '충당부채' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")
except Exception as e:
    print(f"❌ 오류: 데이터 삽입 중 문제가 발생했습니다. 변경사항을 롤백합니다.")
    print(f"오류 내용: {e}")
    conn.rollback()
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

테이블 '충당부채'가 성공적으로 생성되었거나 이미 존재합니다.
✅ '충당부채_통합테이블_2014-2023.csv' 파일의 데이터가 '충당부채' 테이블에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [83]:
import sqlite3
import csv
import os

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/확정급여제도 관련 비용의 계정과목별 금액_통합테이블_2014-2023.csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

def convert_to_float_or_none(value):
    """
    Converts a string to a float, handling commas, spaces, and empty strings as None.
    """
    if value is None or value.strip() == '':
        return None
    try:
        # Remove commas and spaces before conversion
        cleaned_value = value.replace(',', '').replace('"', '').strip()
        return float(cleaned_value)
    except (ValueError, IndexError):
        return None

try:
    # Create a new table for the defined benefit plan-related costs
    create_table_query = """
    CREATE TABLE IF NOT EXISTS 확정급여제도_비용 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        연도 INTEGER NOT NULL,
        변동내역 TEXT,
        단위 TEXT,
        매출원가 REAL,
        판매비와관리비_등 REAL
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 '확정급여제도_비용'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO 확정급여제도_비용 (
            연도,
            변동내역,
            단위,
            매출원가,
            판매비와관리비_등
        ) VALUES (?, ?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                year = int(row[0])
                column_name = row[1]
                unit = row[2]
                cogs = convert_to_float_or_none(row[3])
                sga = convert_to_float_or_none(row[4])

                # Execute the insert query
                cursor.execute(insert_query, (
                    year,
                    column_name,
                    unit,
                    cogs,
                    sga
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{os.path.basename(csv_file_path)}' 파일의 데이터가 '확정급여제도_비용' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")

테이블 '확정급여제도_비용'가 성공적으로 생성되었거나 이미 존재합니다.
✅ '확정급여제도 관련 비용의 계정과목별 금액_통합테이블_2014-2023.csv' 파일의 데이터가 '확정급여제도_비용' 테이블에 성공적으로 삽입되었습니다.


In [84]:
import sqlite3
import csv
import os

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/확정급여제도_관련_비용_통합테이블_2014-2023.csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

def convert_to_float_or_none(value):
    """
    Converts a string to a float, handling commas, spaces, and empty strings as None.
    """
    if value is None or value.strip() == '':
        return None
    try:
        # Remove commas and spaces before conversion
        cleaned_value = value.replace(',', '').replace('"', '').strip()
        return float(cleaned_value)
    except (ValueError, IndexError):
        return None

try:
    # Create a new table for the defined benefit plan-related costs
    create_table_query = """
    CREATE TABLE IF NOT EXISTS 확정급여제도_관련_비용 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        연도 INTEGER NOT NULL,
        변동내역 TEXT,
        단위 TEXT,
        과거근무원가 REAL,
        당기근무원가 REAL,
        순이자수익 REAL,
        순이자원가 REAL
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 '확정급여제도_관련_비용'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO 확정급여제도_관련_비용 (
            연도,
            변동내역,
            단위,
            과거근무원가,
            당기근무원가,
            순이자수익,
            순이자원가
        ) VALUES (?, ?, ?, ?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                year = int(row[0])
                column_name = row[1]
                unit = row[2]
                past_service_cost = convert_to_float_or_none(row[3])
                current_service_cost = convert_to_float_or_none(row[4])
                net_interest_income = convert_to_float_or_none(row[5])
                net_interest_cost = convert_to_float_or_none(row[6])

                # Execute the insert query
                cursor.execute(insert_query, (
                    year,
                    column_name,
                    unit,
                    past_service_cost,
                    current_service_cost,
                    net_interest_income,
                    net_interest_cost
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{os.path.basename(csv_file_path)}' 파일의 데이터가 '확정급여제도_관련_비용' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")
except Exception as e:
    print(f"❌ 오류: 데이터 삽입 중 문제가 발생했습니다. 변경사항을 롤백합니다.")
    print(f"오류 내용: {e}")
    conn.rollback()
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

테이블 '확정급여제도_관련_비용'가 성공적으로 생성되었거나 이미 존재합니다.
✅ '확정급여제도_관련_비용_통합테이블_2014-2023.csv' 파일의 데이터가 '확정급여제도_관련_비용' 테이블에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [85]:
import sqlite3
import csv
import os

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/확정급여채무_변동내역_통합테이블_2014-2023.csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

def convert_to_float_or_none(value):
    """
    Converts a string to a float, handling commas, spaces, and empty strings as None.
    """
    if value is None or value.strip() == '':
        return None
    try:
        # Remove commas and spaces before conversion
        cleaned_value = value.replace(',', '').replace('"', '').strip()
        return float(cleaned_value)
    except (ValueError, IndexError):
        return None

try:
    # Create a new table for the defined benefit obligation movements
    create_table_query = """
    CREATE TABLE IF NOT EXISTS 확정급여채무_변동내역 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        연도 INTEGER NOT NULL,
        항목 TEXT,
        변동내역 TEXT,
        금액 REAL,
        단위 TEXT,
        값_유형 TEXT
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 '확정급여채무_변동내역'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO 확정급여채무_변동내역 (
            연도,
            항목,
            변동내역,
            금액,
            단위,
            값_유형
        ) VALUES (?, ?, ?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                year = int(row[0])
                line_item = row[1]
                column_name = row[2]
                amount = convert_to_float_or_none(row[3])
                unit = row[4]
                value_type = row[5]

                # Execute the insert query
                cursor.execute(insert_query, (
                    year,
                    line_item,
                    column_name,
                    amount,
                    unit,
                    value_type
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{os.path.basename(csv_file_path)}' 파일의 데이터가 '확정급여채무_변동내역' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")
except Exception as e:
    print(f"❌ 오류: 데이터 삽입 중 문제가 발생했습니다. 변경사항을 롤백합니다.")
    print(f"오류 내용: {e}")
    conn.rollback()
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

테이블 '확정급여채무_변동내역'가 성공적으로 생성되었거나 이미 존재합니다.
✅ '확정급여채무_변동내역_통합테이블_2014-2023.csv' 파일의 데이터가 '확정급여채무_변동내역' 테이블에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [86]:
import sqlite3
import csv
import os

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/확정급여채무_통합테이블_2014-2023.csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

def convert_to_float_or_none(value):
    """
    Converts a string to a float, handling commas, spaces, and empty strings as None.
    """
    if value is None or value.strip() == '':
        return None
    try:
        # Remove commas and spaces before conversion
        cleaned_value = value.replace(',', '').replace('"', '').strip()
        return float(cleaned_value)
    except (ValueError, IndexError):
        return None

try:
    # Create a new table for the defined benefit obligation data
    create_table_query = """
    CREATE TABLE IF NOT EXISTS 확정급여채무_통합 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        연도 INTEGER NOT NULL,
        변동내역 TEXT,
        단위 TEXT,
        기금_미적립_현재가치 REAL,
        기금_적립_현재가치 REAL,
        사외적립자산_공정가치 REAL
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 '확정급여채무_통합'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO 확정급여채무_통합 (
            연도,
            변동내역,
            단위,
            기금_미적립_현재가치,
            기금_적립_현재가치,
            사외적립자산_공정가치
        ) VALUES (?, ?, ?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                year = int(row[0])
                column_name = row[1]
                unit = row[2]
                unfunded_pvo = convert_to_float_or_none(row[3])
                funded_pvo = convert_to_float_or_none(row[4])
                plan_assets_fv = convert_to_float_or_none(row[5])

                # Execute the insert query
                cursor.execute(insert_query, (
                    year,
                    column_name,
                    unit,
                    unfunded_pvo,
                    funded_pvo,
                    plan_assets_fv
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{os.path.basename(csv_file_path)}' 파일의 데이터가 '확정급여채무_통합' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")
except Exception as e:
    print(f"❌ 오류: 데이터 삽입 중 문제가 발생했습니다. 변경사항을 롤백합니다.")
    print(f"오류 내용: {e}")
    conn.rollback()
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

테이블 '확정급여채무_통합'가 성공적으로 생성되었거나 이미 존재합니다.
✅ '확정급여채무_통합테이블_2014-2023.csv' 파일의 데이터가 '확정급여채무_통합' 테이블에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [87]:
import sqlite3

# 확인하고 싶은 테이블 목록
tables = [
    '매각예정분류',
    '무상할당_배출권',
    '사외적립자산',
    '확정급여채무_민감도분석',
    '보험수리적_가정',
    '충당부채' ,
    '확정급여제도_비용',
    '확정급여제도_관련_비용',
    '확정급여채무_변동내역',
    '확정급여채무_통합'
]

# 데이터베이스에 연결
conn = sqlite3.connect('samsung_audit_report.db')
cursor = conn.cursor()

try:
    print("=== 데이터베이스 테이블 확인 ===")
    for table_name in tables:
        print(f"\n--- 테이블: {table_name} ---")
        
        # 총 행의 개수 확인
        count_query = f"SELECT COUNT(*) FROM {table_name};"
        cursor.execute(count_query)
        count = cursor.fetchone()[0]
        print(f"총 데이터 개수: {count}개")
        
        # 상위 5개 행 조회
        select_query = f"SELECT * FROM {table_name} LIMIT 5;"
        cursor.execute(select_query)
        rows = cursor.fetchall()

        if rows:
            # 컬럼 이름 가져오기
            columns = [description[0] for description in cursor.description]
            print(f"컬럼: {', '.join(columns)}")
            
            # 데이터 출력
            for row in rows:
                print(row)
        else:
            print("데이터가 없습니다.")

except sqlite3.OperationalError as e:
    print(f"오류: {e}")
    print("테이블 이름이 올바르지 않거나, 테이블이 존재하지 않습니다.")

finally:
    # 연결 종료
    cursor.close()
    conn.close()
    print("\n=== 확인 완료 ===")

=== 데이터베이스 테이블 확인 ===

--- 테이블: 매각예정분류 ---
총 데이터 개수: 9개
컬럼: id, 연도, 항목, 변동내역, 금액, 단위, 값_유형
(1, 2014, '매각예정분류자산 / 매출채권', '당기말', 49192.0, '백만원', 'number')
(2, 2014, '매각예정분류자산 / 재고자산', '당기말', 5928.0, '백만원', 'number')
(3, 2014, '매각예정분류자산 / 기타유동자산', '당기말', 1578.0, '백만원', 'number')
(4, 2014, '매각예정분류자산 / 유형자산', '당기말', 15150.0, '백만원', 'number')
(5, 2014, '매각예정분류자산 / 투자자산', '당기말', 389558.0, '백만원', 'number')

--- 테이블: 무상할당_배출권 ---
총 데이터 개수: 9개
컬럼: id, 연도, 변동내역, 단위, 무상할당_배출권, 배출량_추정치
(1, 2015, '당기말', '톤', 5876732.0, 6594207.0)
(2, 2016, '당기말', '만톤', 657.0, 743.0)
(3, 2017, '당기말', '만톤', 750.0, 821.0)
(4, 2018, '당기말', '만톤', 711.0, 1131.0)
(5, 2019, '당기말', '만톤', 781.0, 1121.0)

--- 테이블: 사외적립자산 ---
총 데이터 개수: 10개
컬럼: id, 연도, 변동내역, 단위, 급여지급액, 기말, 기초, 기타, 이자수익, 재측정요소
(1, 2014, '당기', '백만원', -135047.0, 6136995.0, 3071581.0, -3199.0, 159655.0, -66114.0)
(2, 2015, '당기', '백만원', -244080.0, 6179950.0, 6136995.0, -8643.0, 266387.0, -116709.0)
(3, 2016, '당기', '백만원', -336449.0, 6253751.0, 6179950.0, -35187.0, 237

# 선유 CSV

In [88]:
import sqlite3
import csv
import os

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/[매도가능금융자산=공정가치금융자산_변동내역]matrix_long_scaled.csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

def convert_to_float_or_none(value):
    """
    Converts a string to a float, handling commas, spaces, and empty strings as None.
    """
    if value is None or value.strip() == '':
        return None
    try:
        # Remove commas and spaces before conversion
        cleaned_value = value.replace(',', '').replace('"', '').strip()
        return float(cleaned_value)
    except (ValueError, IndexError):
        return None

try:
    # Create a new table for the available-for-sale financial assets data
    create_table_query = """
    CREATE TABLE IF NOT EXISTS 매도가능금융자산_변동내역 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        연도 INTEGER NOT NULL,
        항목 TEXT,
        변동내역 TEXT,
        금액 REAL,
        단위 TEXT,
        단위_배수 REAL,
        조정된_금액 REAL
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 '매도가능금융자산_변동내역'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO 매도가능금융자산_변동내역 (
            연도,
            항목,
            변동내역,
            금액,
            단위,
            단위_배수,
            조정된_금액
        ) VALUES (?, ?, ?, ?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                year = int(row[0])
                line_item = row[1]
                column_name = row[2]
                amount = convert_to_float_or_none(row[3])
                unit = row[4]
                unit_multiplier = convert_to_float_or_none(row[5])
                amount_scaled = convert_to_float_or_none(row[6])

                # Execute the insert query
                cursor.execute(insert_query, (
                    year,
                    line_item,
                    column_name,
                    amount,
                    unit,
                    unit_multiplier,
                    amount_scaled
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{os.path.basename(csv_file_path)}' 파일의 데이터가 '매도가능금융자산_변동내역' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")
except Exception as e:
    print(f"❌ 오류: 데이터 삽입 중 문제가 발생했습니다. 변경사항을 롤백합니다.")
    print(f"오류 내용: {e}")
    conn.rollback()
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

테이블 '매도가능금융자산_변동내역'가 성공적으로 생성되었거나 이미 존재합니다.
✅ '[매도가능금융자산=공정가치금융자산_변동내역]matrix_long_scaled.csv' 파일의 데이터가 '매도가능금융자산_변동내역' 테이블에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [89]:
import sqlite3
import csv
import os

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/[매도가능금융자산=공정가치금융자산_상장주식]matrix_long_scaled.csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

def convert_to_float_or_none(value):
    """
    Converts a string to a float, handling commas, spaces, and empty strings as None.
    """
    if value is None or value.strip() == '':
        return None
    try:
        # Remove commas and spaces before conversion
        cleaned_value = value.replace(',', '').replace('"', '').strip()
        return float(cleaned_value)
    except (ValueError, IndexError):
        return None

try:
    # Create a new table for the listed stocks data
    create_table_query = """
    CREATE TABLE IF NOT EXISTS 매도가능금융자산_상장주식 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        연도 INTEGER NOT NULL,
        항목 TEXT,
        변동내역 TEXT,
        금액 REAL,
        단위 TEXT,
        단위_배수 REAL,
        조정된_금액 REAL
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 '매도가능금융자산_상장주식'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO 매도가능금융자산_상장주식 (
            연도,
            항목,
            변동내역,
            금액,
            단위,
            단위_배수,
            조정된_금액
        ) VALUES (?, ?, ?, ?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                year = int(row[0])
                line_item = row[1]
                column_name = row[2]
                amount = convert_to_float_or_none(row[3])
                unit = row[4]
                unit_multiplier = convert_to_float_or_none(row[5])
                amount_scaled = convert_to_float_or_none(row[6])

                # Execute the insert query
                cursor.execute(insert_query, (
                    year,
                    line_item,
                    column_name,
                    amount,
                    unit,
                    unit_multiplier,
                    amount_scaled
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{os.path.basename(csv_file_path)}' 파일의 데이터가 '매도가능금융자산_상장주식' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")
except Exception as e:
    print(f"❌ 오류: 데이터 삽입 중 문제가 발생했습니다. 변경사항을 롤백합니다.")
    print(f"오류 내용: {e}")
    conn.rollback()
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

테이블 '매도가능금융자산_상장주식'가 성공적으로 생성되었거나 이미 존재합니다.
✅ '[매도가능금융자산=공정가치금융자산_상장주식]matrix_long_scaled.csv' 파일의 데이터가 '매도가능금융자산_상장주식' 테이블에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [90]:
import sqlite3
import csv
import os

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/[매출채권_할인내역]matrix_long_scaled.csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

def convert_to_float_or_none(value):
    """
    Converts a string to a float, handling commas, spaces, and empty strings as None.
    """
    if value is None or value.strip() == '':
        return None
    try:
        # Remove commas and spaces before conversion
        cleaned_value = value.replace(',', '').replace('"', '').strip()
        return float(cleaned_value)
    except (ValueError, IndexError):
        return None

try:
    # Create a new table for the account receivables and discount details
    create_table_query = """
    CREATE TABLE IF NOT EXISTS 매출채권_할인내역 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        연도 INTEGER NOT NULL,
        항목 TEXT,
        변동내역 TEXT,
        금액 REAL,
        단위 TEXT,
        단위_배수 REAL,
        조정된_금액 REAL
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 '매출채권_할인내역'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO 매출채권_할인내역 (
            연도,
            항목,
            변동내역,
            금액,
            단위,
            단위_배수,
            조정된_금액
        ) VALUES (?, ?, ?, ?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                year = int(row[0])
                line_item = row[1]
                column_name = row[2]
                amount = convert_to_float_or_none(row[3])
                unit = row[4]
                unit_multiplier = convert_to_float_or_none(row[5])
                amount_scaled = convert_to_float_or_none(row[6])

                # Execute the insert query
                cursor.execute(insert_query, (
                    year,
                    line_item,
                    column_name,
                    amount,
                    unit,
                    unit_multiplier,
                    amount_scaled
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{os.path.basename(csv_file_path)}' 파일의 데이터가 '매출채권_할인내역' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")
except Exception as e:
    print(f"❌ 오류: 데이터 삽입 중 문제가 발생했습니다. 변경사항을 롤백합니다.")
    print(f"오류 내용: {e}")
    conn.rollback()
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

테이블 '매출채권_할인내역'가 성공적으로 생성되었거나 이미 존재합니다.
✅ '[매출채권_할인내역]matrix_long_scaled.csv' 파일의 데이터가 '매출채권_할인내역' 테이블에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [91]:
import sqlite3
import csv
import os

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/[매출채권및미수금_내역]matrix_long_scaled.csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

def convert_to_float_or_none(value):
    """
    Converts a string to a float, handling commas, spaces, and empty strings as None.
    """
    if value is None or value.strip() == '':
        return None
    try:
        # Remove commas and spaces before conversion
        cleaned_value = value.replace(',', '').replace('"', '').strip()
        return float(cleaned_value)
    except (ValueError, IndexError):
        return None

try:
    # Create a new table for the accounts receivable and other receivables data
    create_table_query = """
    CREATE TABLE IF NOT EXISTS 매출채권과미수금 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        연도 INTEGER NOT NULL,
        항목 TEXT,
        변동내역 TEXT,
        금액 REAL,
        단위 TEXT,
        단위_배수 REAL,
        조정된_금액 REAL
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 '매출채권과미수금'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO 매출채권과미수금 (
            연도,
            항목,
            변동내역,
            금액,
            단위,
            단위_배수,
            조정된_금액
        ) VALUES (?, ?, ?, ?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                year = int(row[0])
                line_item = row[1]
                column_name = row[2]
                amount = convert_to_float_or_none(row[3])
                unit = row[4]
                unit_multiplier = convert_to_float_or_none(row[5])
                amount_scaled = convert_to_float_or_none(row[6])

                # Execute the insert query
                cursor.execute(insert_query, (
                    year,
                    line_item,
                    column_name,
                    amount,
                    unit,
                    unit_multiplier,
                    amount_scaled
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{os.path.basename(csv_file_path)}' 파일의 데이터가 '매출채권과미수금' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")
except Exception as e:
    print(f"❌ 오류: 데이터 삽입 중 문제가 발생했습니다. 변경사항을 롤백합니다.")
    print(f"오류 내용: {e}")
    conn.rollback()
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

테이블 '매출채권과미수금'가 성공적으로 생성되었거나 이미 존재합니다.
✅ '[매출채권및미수금_내역]matrix_long_scaled.csv' 파일의 데이터가 '매출채권과미수금' 테이블에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [92]:
import sqlite3
import csv
import os

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/[매출채권및미수금_연체일구분]matrix_long_scaled.csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

def convert_to_float_or_none(value):
    """
    Converts a string to a float, handling commas, spaces, and empty strings as None.
    """
    if value is None or value.strip() == '':
        return None
    try:
        # Remove commas and spaces before conversion
        cleaned_value = value.replace(',', '').replace('"', '').strip()
        return float(cleaned_value)
    except (ValueError, IndexError):
        return None

try:
    # Create a new table for the account receivables and other receivables by aging
    create_table_query = """
    CREATE TABLE IF NOT EXISTS 매출채권과미수금_연체일구분 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        연도 INTEGER NOT NULL,
        항목 TEXT,
        변동내역 TEXT,
        금액 REAL,
        단위 TEXT,
        단위_배수 REAL,
        조정된_금액 REAL
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 '매출채권과미수금_연체일구분'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO 매출채권과미수금_연체일구분 (
            연도,
            항목,
            변동내역,
            금액,
            단위,
            단위_배수,
            조정된_금액
        ) VALUES (?, ?, ?, ?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                year = int(row[0])
                line_item = row[1]
                column_name = row[2]
                amount = convert_to_float_or_none(row[3])
                unit = row[4]
                unit_multiplier = convert_to_float_or_none(row[5])
                amount_scaled = convert_to_float_or_none(row[6])

                # Execute the insert query
                cursor.execute(insert_query, (
                    year,
                    line_item,
                    column_name,
                    amount,
                    unit,
                    unit_multiplier,
                    amount_scaled
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{os.path.basename(csv_file_path)}' 파일의 데이터가 '매출채권과미수금_연체일구분' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")
except Exception as e:
    print(f"❌ 오류: 데이터 삽입 중 문제가 발생했습니다. 변경사항을 롤백합니다.")
    print(f"오류 내용: {e}")
    conn.rollback()
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

테이블 '매출채권과미수금_연체일구분'가 성공적으로 생성되었거나 이미 존재합니다.
✅ '[매출채권및미수금_연체일구분]matrix_long_scaled.csv' 파일의 데이터가 '매출채권과미수금_연체일구분' 테이블에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [93]:
import sqlite3
import csv
import os

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/[매출채권및미수금_충당금변동내역]matrix_long_scaled.csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

def convert_to_float_or_none(value):
    """
    Converts a string to a float, handling commas, spaces, and empty strings as None.
    """
    if value is None or value.strip() == '':
        return None
    try:
        # Remove commas and spaces before conversion
        cleaned_value = value.replace(',', '').replace('"', '').strip()
        return float(cleaned_value)
    except (ValueError, IndexError):
        return None

try:
    # Create a new table for the accounts receivable and other receivables allowance movements
    create_table_query = """
    CREATE TABLE IF NOT EXISTS 매출채권과미수금_충당금변동내역 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        연도 INTEGER NOT NULL,
        항목 TEXT,
        변동내역 TEXT,
        금액 REAL,
        단위 TEXT,
        단위_배수 REAL,
        조정된_금액 REAL
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 '매출채권과미수금_충당금변동내역'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO 매출채권과미수금_충당금변동내역 (
            연도,
            항목,
            변동내역,
            금액,
            단위,
            단위_배수,
            조정된_금액
        ) VALUES (?, ?, ?, ?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                year = int(row[0])
                line_item = row[1]
                column_name = row[2]
                amount = convert_to_float_or_none(row[3])
                unit = row[4]
                unit_multiplier = convert_to_float_or_none(row[5])
                amount_scaled = convert_to_float_or_none(row[6])

                # Execute the insert query
                cursor.execute(insert_query, (
                    year,
                    line_item,
                    column_name,
                    amount,
                    unit,
                    unit_multiplier,
                    amount_scaled
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{os.path.basename(csv_file_path)}' 파일의 데이터가 '매출채권과미수금_충당금변동내역' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")
except Exception as e:
    print(f"❌ 오류: 데이터 삽입 중 문제가 발생했습니다. 변경사항을 롤백합니다.")
    print(f"오류 내용: {e}")
    conn.rollback()
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

테이블 '매출채권과미수금_충당금변동내역'가 성공적으로 생성되었거나 이미 존재합니다.
✅ '[매출채권및미수금_충당금변동내역]matrix_long_scaled.csv' 파일의 데이터가 '매출채권과미수금_충당금변동내역' 테이블에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [94]:
import sqlite3
import csv
import os

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/[범주별금융상품_금융부채]matrix_long_scaled.csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

def convert_to_float_or_none(value):
    """
    Converts a string to a float, handling commas, spaces, and empty strings as None.
    """
    if value is None or value.strip() == '':
        return None
    try:
        # Remove commas and spaces before conversion
        cleaned_value = value.replace(',', '').replace('"', '').strip()
        return float(cleaned_value)
    except (ValueError, IndexError):
        return None

try:
    # Create a new table for the financial liabilities by category
    create_table_query = """
    CREATE TABLE IF NOT EXISTS 범주별금융상품_금융부채 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        연도 INTEGER NOT NULL,
        항목 TEXT,
        변동내역 TEXT,
        금액 REAL,
        단위 TEXT,
        단위_배수 REAL,
        조정된_금액 REAL
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 '범주별금융상품_금융부채'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO 범주별금융상품_금융부채 (
            연도,
            항목,
            변동내역,
            금액,
            단위,
            단위_배수,
            조정된_금액
        ) VALUES (?, ?, ?, ?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                year = int(row[0])
                line_item = row[1]
                column_name = row[2]
                amount = convert_to_float_or_none(row[3])
                unit = row[4]
                unit_multiplier = convert_to_float_or_none(row[5])
                amount_scaled = convert_to_float_or_none(row[6])

                # Execute the insert query
                cursor.execute(insert_query, (
                    year,
                    line_item,
                    column_name,
                    amount,
                    unit,
                    unit_multiplier,
                    amount_scaled
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{os.path.basename(csv_file_path)}' 파일의 데이터가 '범주별금융상품_금융부채' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")
except Exception as e:
    print(f"❌ 오류: 데이터 삽입 중 문제가 발생했습니다. 변경사항을 롤백합니다.")
    print(f"오류 내용: {e}")
    conn.rollback()
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

테이블 '범주별금융상품_금융부채'가 성공적으로 생성되었거나 이미 존재합니다.
✅ '[범주별금융상품_금융부채]matrix_long_scaled.csv' 파일의 데이터가 '범주별금융상품_금융부채' 테이블에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [95]:
import sqlite3
import csv
import os

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/[범주별금융상품_금융자산]matrix_long_scaled.csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

def convert_to_float_or_none(value):
    """
    Converts a string to a float, handling commas, spaces, and empty strings as None.
    """
    if value is None or value.strip() == '':
        return None
    try:
        # Remove commas and spaces before conversion
        cleaned_value = value.replace(',', '').replace('"', '').strip()
        return float(cleaned_value)
    except (ValueError, IndexError):
        return None

try:
    # Create a new table for the financial assets by category
    create_table_query = """
    CREATE TABLE IF NOT EXISTS 범주별금융상품_금융자산 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        연도 INTEGER NOT NULL,
        항목 TEXT,
        변동내역 TEXT,
        금액 REAL,
        단위 TEXT,
        단위_배수 REAL,
        조정된_금액 REAL
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 '범주별금융상품_금융자산'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO 범주별금융상품_금융자산 (
            연도,
            항목,
            변동내역,
            금액,
            단위,
            단위_배수,
            조정된_금액
        ) VALUES (?, ?, ?, ?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                year = int(row[0])
                line_item = row[1]
                column_name = row[2]
                amount = convert_to_float_or_none(row[3])
                unit = row[4]
                unit_multiplier = convert_to_float_or_none(row[5])
                amount_scaled = convert_to_float_or_none(row[6])

                # Execute the insert query
                cursor.execute(insert_query, (
                    year,
                    line_item,
                    column_name,
                    amount,
                    unit,
                    unit_multiplier,
                    amount_scaled
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{os.path.basename(csv_file_path)}' 파일의 데이터가 '범주별금융상품_금융자산' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")
except Exception as e:
    print(f"❌ 오류: 데이터 삽입 중 문제가 발생했습니다. 변경사항을 롤백합니다.")
    print(f"오류 내용: {e}")
    conn.rollback()
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

테이블 '범주별금융상품_금융자산'가 성공적으로 생성되었거나 이미 존재합니다.
✅ '[범주별금융상품_금융자산]matrix_long_scaled.csv' 파일의 데이터가 '범주별금융상품_금융자산' 테이블에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [96]:
import sqlite3
import csv
import os

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/[범주별순손익구분_금융부채]matrix_long_scaled.csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

def convert_to_float_or_none(value):
    """
    Converts a string to a float, handling commas, spaces, and empty strings as None.
    """
    if value is None or value.strip() == '':
        return None
    try:
        # Remove commas and spaces before conversion
        cleaned_value = value.replace(',', '').replace('"', '').strip()
        return float(cleaned_value)
    except (ValueError, IndexError):
        return None

try:
    # Create a new table for the financial liabilities by category and net income
    create_table_query = """
    CREATE TABLE IF NOT EXISTS 범주별순손익구분_금융부채 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        연도 INTEGER NOT NULL,
        항목 TEXT,
        변동내역 TEXT,
        금액 REAL,
        단위 TEXT,
        단위_배수 REAL,
        조정된_금액 REAL
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 '범주별순손익구분_금융부채'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO 범주별순손익구분_금융부채 (
            연도,
            항목,
            변동내역,
            금액,
            단위,
            단위_배수,
            조정된_금액
        ) VALUES (?, ?, ?, ?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                year = int(row[0])
                line_item = row[1]
                column_name = row[2]
                amount = convert_to_float_or_none(row[3])
                unit = row[4]
                unit_multiplier = convert_to_float_or_none(row[5])
                amount_scaled = convert_to_float_or_none(row[6])

                # Execute the insert query
                cursor.execute(insert_query, (
                    year,
                    line_item,
                    column_name,
                    amount,
                    unit,
                    unit_multiplier,
                    amount_scaled
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{os.path.basename(csv_file_path)}' 파일의 데이터가 '범주별순손익구분_금융부채' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")
except Exception as e:
    print(f"❌ 오류: 데이터 삽입 중 문제가 발생했습니다. 변경사항을 롤백합니다.")
    print(f"오류 내용: {e}")
    conn.rollback()
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

테이블 '범주별순손익구분_금융부채'가 성공적으로 생성되었거나 이미 존재합니다.
✅ '[범주별순손익구분_금융부채]matrix_long_scaled.csv' 파일의 데이터가 '범주별순손익구분_금융부채' 테이블에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [97]:
import sqlite3
import csv
import os

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/[범주별순손익구분_금융부채]matrix_long_scaled.csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

def convert_to_float_or_none(value):
    """
    Converts a string to a float, handling commas, spaces, and empty strings as None.
    """
    if value is None or value.strip() == '':
        return None
    try:
        # Remove commas and spaces before conversion
        cleaned_value = value.replace(',', '').replace('"', '').strip()
        return float(cleaned_value)
    except (ValueError, IndexError):
        return None

try:
    # Create a new table for the financial liabilities by category and net income
    create_table_query = """
    CREATE TABLE IF NOT EXISTS 범주별순손익구분_금융부채 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        연도 INTEGER NOT NULL,
        항목 TEXT,
        변동내역 TEXT,
        금액 REAL,
        단위 TEXT,
        단위_배수 REAL,
        조정된_금액 REAL
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 '범주별순손익구분_금융부채'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO 범주별순손익구분_금융부채 (
            연도,
            항목,
            변동내역,
            금액,
            단위,
            단위_배수,
            조정된_금액
        ) VALUES (?, ?, ?, ?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                year = int(row[0])
                line_item = row[1]
                column_name = row[2]
                amount = convert_to_float_or_none(row[3])
                unit = row[4]
                unit_multiplier = convert_to_float_or_none(row[5])
                amount_scaled = convert_to_float_or_none(row[6])

                # Execute the insert query
                cursor.execute(insert_query, (
                    year,
                    line_item,
                    column_name,
                    amount,
                    unit,
                    unit_multiplier,
                    amount_scaled
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{os.path.basename(csv_file_path)}' 파일의 데이터가 '범주별순손익구분_금융부채' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")
except Exception as e:
    print(f"❌ 오류: 데이터 삽입 중 문제가 발생했습니다. 변경사항을 롤백합니다.")
    print(f"오류 내용: {e}")
    conn.rollback()
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

테이블 '범주별순손익구분_금융부채'가 성공적으로 생성되었거나 이미 존재합니다.
✅ '[범주별순손익구분_금융부채]matrix_long_scaled.csv' 파일의 데이터가 '범주별순손익구분_금융부채' 테이블에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [98]:
import sqlite3
import csv
import os

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/[범주별순손익구분_금융자산]matrix_long_scaled.csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

def convert_to_float_or_none(value):
    """
    Converts a string to a float, handling commas, spaces, and empty strings as None.
    """
    if value is None or value.strip() == '':
        return None
    try:
        # Remove commas and spaces before conversion
        cleaned_value = value.replace(',', '').replace('"', '').strip()
        return float(cleaned_value)
    except (ValueError, IndexError):
        return None

try:
    # Create a new table for the financial assets by category and net income
    create_table_query = """
    CREATE TABLE IF NOT EXISTS 범주별순손익구분_금융자산 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        연도 INTEGER NOT NULL,
        항목 TEXT,
        변동내역 TEXT,
        금액 REAL,
        단위 TEXT,
        단위_배수 REAL,
        조정된_금액 REAL
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 '범주별순손익구분_금융자산'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO 범주별순손익구분_금융자산 (
            연도,
            항목,
            변동내역,
            금액,
            단위,
            단위_배수,
            조정된_금액
        ) VALUES (?, ?, ?, ?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                year = int(row[0])
                line_item = row[1]
                column_name = row[2]
                amount = convert_to_float_or_none(row[3])
                unit = row[4]
                unit_multiplier = convert_to_float_or_none(row[5])
                amount_scaled = convert_to_float_or_none(row[6])

                # Execute the insert query
                cursor.execute(insert_query, (
                    year,
                    line_item,
                    column_name,
                    amount,
                    unit,
                    unit_multiplier,
                    amount_scaled
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{os.path.basename(csv_file_path)}' 파일의 데이터가 '범주별순손익구분_금융자산' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")
except Exception as e:
    print(f"❌ 오류: 데이터 삽입 중 문제가 발생했습니다. 변경사항을 롤백합니다.")
    print(f"오류 내용: {e}")
    conn.rollback()
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

테이블 '범주별순손익구분_금융자산'가 성공적으로 생성되었거나 이미 존재합니다.
✅ '[범주별순손익구분_금융자산]matrix_long_scaled.csv' 파일의 데이터가 '범주별순손익구분_금융자산' 테이블에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [99]:
import sqlite3
import csv
import os

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/[사채내역]matrix_long_scaled.csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

def convert_to_float_or_none(value):
    """
    Converts a string to a float, handling commas, spaces, and empty strings as None.
    """
    if value is None or value.strip() == '':
        return None
    try:
        # Remove commas and spaces before conversion
        cleaned_value = value.replace(',', '').replace('"', '').strip()
        # Handle parentheses for negative numbers, if any
        if cleaned_value.startswith('(') and cleaned_value.endswith(')'):
            cleaned_value = '-' + cleaned_value[1:-1]
        return float(cleaned_value)
    except (ValueError, IndexError):
        return None

try:
    # Create a new table for the bond details data
    create_table_query = """
    CREATE TABLE IF NOT EXISTS 사채내역 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        연도 INTEGER NOT NULL,
        항목 TEXT,
        변동내역 TEXT,
        금액 REAL,
        단위 TEXT,
        단위_배수 REAL,
        조정된_금액 REAL
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 '사채내역'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO 사채내역 (
            연도,
            항목,
            변동내역,
            금액,
            단위,
            단위_배수,
            조정된_금액
        ) VALUES (?, ?, ?, ?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                year = int(row[0])
                line_item = row[1]
                column_name = row[2]
                amount = convert_to_float_or_none(row[3])
                unit = row[4]
                unit_multiplier = convert_to_float_or_none(row[5])
                amount_scaled = convert_to_float_or_none(row[6])

                # Execute the insert query
                cursor.execute(insert_query, (
                    year,
                    line_item,
                    column_name,
                    amount,
                    unit,
                    unit_multiplier,
                    amount_scaled
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{os.path.basename(csv_file_path)}' 파일의 데이터가 '사채내역' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")
except Exception as e:
    print(f"❌ 오류: 데이터 삽입 중 문제가 발생했습니다. 변경사항을 롤백합니다.")
    print(f"오류 내용: {e}")
    conn.rollback()
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

테이블 '사채내역'가 성공적으로 생성되었거나 이미 존재합니다.
✅ '[사채내역]matrix_long_scaled.csv' 파일의 데이터가 '사채내역' 테이블에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [100]:
import sqlite3
import csv
import os

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/[사채상환내역]matrix_long_scaled.csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

def convert_to_float_or_none(value):
    """
    Converts a string to a float, handling commas, spaces, and empty strings as None.
    """
    if value is None or value.strip() == '':
        return None
    try:
        # Remove commas and spaces before conversion
        cleaned_value = value.replace(',', '').replace('"', '').strip()
        return float(cleaned_value)
    except (ValueError, IndexError):
        return None

try:
    # Create a new table for the bond repayment details
    create_table_query = """
    CREATE TABLE IF NOT EXISTS 사채상환내역 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        연도 INTEGER NOT NULL,
        항목 TEXT,
        변동내역 TEXT,
        금액 REAL,
        단위 TEXT,
        단위_배수 REAL,
        조정된_금액 REAL
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 '사채상환내역'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO 사채상환내역 (
            연도,
            항목,
            변동내역,
            금액,
            단위,
            단위_배수,
            조정된_금액
        ) VALUES (?, ?, ?, ?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                year = int(row[0])
                line_item = row[1]
                column_name = row[2]
                amount = convert_to_float_or_none(row[3])
                unit = row[4]
                unit_multiplier = convert_to_float_or_none(row[5])
                amount_scaled = convert_to_float_or_none(row[6])

                # Execute the insert query
                cursor.execute(insert_query, (
                    year,
                    line_item,
                    column_name,
                    amount,
                    unit,
                    unit_multiplier,
                    amount_scaled
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{os.path.basename(csv_file_path)}' 파일의 데이터가 '사채상환내역' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")
except Exception as e:
    print(f"❌ 오류: 데이터 삽입 중 문제가 발생했습니다. 변경사항을 롤백합니다.")
    print(f"오류 내용: {e}")
    conn.rollback()
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

테이블 '사채상환내역'가 성공적으로 생성되었거나 이미 존재합니다.
✅ '[사채상환내역]matrix_long_scaled.csv' 파일의 데이터가 '사채상환내역' 테이블에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [101]:
import sqlite3
import csv
import os

# Define the database and CSV file paths
db_name = 'samsung_audit_report.db'
csv_file_path = 'DB_csv/[차입금내역]matrix_long_scaled.csv'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

def convert_to_float_or_none(value):
    """
    Converts a string to a float, handling commas, spaces, and empty strings as None.
    """
    if value is None or value.strip() == '':
        return None
    try:
        # Remove commas and spaces before conversion
        cleaned_value = value.replace(',', '').replace('"', '').strip()
        return float(cleaned_value)
    except (ValueError, IndexError):
        return None

try:
    # Create a new table for the borrowing details data
    create_table_query = """
    CREATE TABLE IF NOT EXISTS 차입금내역 (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        연도 INTEGER NOT NULL,
        항목 TEXT,
        변동내역 TEXT,
        금액 REAL,
        단위 TEXT,
        단위_배수 REAL,
        조정된_금액 REAL
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print(f"테이블 '차입금내역'가 성공적으로 생성되었거나 이미 존재합니다.")

    # Read and insert data from the CSV file
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)

        # SQL INSERT query
        insert_query = """
        INSERT INTO 차입금내역 (
            연도,
            항목,
            변동내역,
            금액,
            단위,
            단위_배수,
            조정된_금액
        ) VALUES (?, ?, ?, ?, ?, ?, ?);
        """

        # Iterate over the CSV rows and insert into the database
        for row in csv_reader:
            try:
                # Type casting and data cleaning
                year = int(row[0])
                line_item = row[1]
                column_name = row[2]
                amount = convert_to_float_or_none(row[3])
                unit = row[4]
                unit_multiplier = convert_to_float_or_none(row[5])
                amount_scaled = convert_to_float_or_none(row[6])

                # Execute the insert query
                cursor.execute(insert_query, (
                    year,
                    line_item,
                    column_name,
                    amount,
                    unit,
                    unit_multiplier,
                    amount_scaled
                ))
            except (ValueError, IndexError) as e:
                print(f"경고: 데이터 변환 또는 행 구조 오류 발생. 해당 행을 건너뜁니다: {row}")
                print(f"오류 내용: {e}")
                continue

    # Commit the changes
    conn.commit()
    print(f"✅ '{os.path.basename(csv_file_path)}' 파일의 데이터가 '차입금내역' 테이블에 성공적으로 삽입되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해 주세요.")
except Exception as e:
    print(f"❌ 오류: 데이터 삽입 중 문제가 발생했습니다. 변경사항을 롤백합니다.")
    print(f"오류 내용: {e}")
    conn.rollback()
finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("데이터베이스 연결이 정상적으로 종료되었습니다.")

테이블 '차입금내역'가 성공적으로 생성되었거나 이미 존재합니다.
✅ '[차입금내역]matrix_long_scaled.csv' 파일의 데이터가 '차입금내역' 테이블에 성공적으로 삽입되었습니다.
데이터베이스 연결이 정상적으로 종료되었습니다.


In [103]:
import sqlite3

# 확인하고 싶은 테이블 목록
tables = [
    '매도가능금융자산_변동내역',
    '매도가능금융자산_상장주식',
    '매출채권_할인내역',
    '매출채권과미수금',
    '매출채권과미수금_연체일구분',
    '매출채권과미수금_충당금변동내역',
    '범주별금융상품_금융자산',
    '범주별금융상품_금융부채',
    '범주별순손익구분_금융자산',
    '범주별순손익구분_금융부채',
    '사채내역',
    '사채상환내역',
    '차입금내역'
]

# 데이터베이스에 연결
conn = sqlite3.connect('samsung_audit_report.db')
cursor = conn.cursor()

try:
    print("=== 데이터베이스 테이블 확인 ===")
    for table_name in tables:
        print(f"\n--- 테이블: {table_name} ---")
        
        # 총 행의 개수 확인
        count_query = f"SELECT COUNT(*) FROM {table_name};"
        cursor.execute(count_query)
        count = cursor.fetchone()[0]
        print(f"총 데이터 개수: {count}개")
        
        # 상위 5개 행 조회
        select_query = f"SELECT * FROM {table_name} LIMIT 5;"
        cursor.execute(select_query)
        rows = cursor.fetchall()

        if rows:
            # 컬럼 이름 가져오기
            columns = [description[0] for description in cursor.description]
            print(f"컬럼: {', '.join(columns)}")
            
            # 데이터 출력
            for row in rows:
                print(row)
        else:
            print("데이터가 없습니다.")

except sqlite3.OperationalError as e:
    print(f"오류: {e}")
    print("테이블 이름이 올바르지 않거나, 테이블이 존재하지 않습니다.")

finally:
    # 연결 종료
    cursor.close()
    conn.close()
    print("\n=== 확인 완료 ===")

=== 데이터베이스 테이블 확인 ===

--- 테이블: 매도가능금융자산_변동내역 ---
총 데이터 개수: 70개
컬럼: id, 연도, 항목, 변동내역, 금액, 단위, 단위_배수, 조정된_금액
(1, 2014, '기초금액', '당기', 5445479.0, '백만원', 1.0, 5445479.0)
(2, 2014, '취득', '당기', 5602199.0, '백만원', 1.0, 5602199.0)
(3, 2014, '처분', '당기', -2351963.0, '백만원', 1.0, -2351963.0)
(4, 2014, '매도가능금융자산평가손익', '당기', -514077.0, '백만원', 1.0, -514077.0)
(5, 2014, '손상', '당기', -53053.0, '백만원', 1.0, -53053.0)

--- 테이블: 매도가능금융자산_상장주식 ---
총 데이터 개수: 648개
컬럼: id, 연도, 항목, 변동내역, 금액, 단위, 단위_배수, 조정된_금액
(1, 2014, '삼성SDI', '당기말 / 보유주식수(주)', 13462673.0, '백만원', 1.0, 13462673.0)
(2, 2014, '삼성SDI', '당기말 / 지분율(%)', 19.6, '백만원', 1.0, 19.6)
(3, 2014, '삼성SDI', '당기말 / 취득원가', 1542857.0, '백만원', 1.0, 1542857.0)
(4, 2014, '삼성SDI', '당기말 / 장부가액 (시장가치)', 1561670.0, '백만원', 1.0, 1561670.0)
(5, 2014, '삼성중공업', '당기말 / 보유주식수(주)', 40675641.0, '백만원', 1.0, 40675641.0)

--- 테이블: 매출채권_할인내역 ---
총 데이터 개수: 264개
컬럼: id, 연도, 항목, 변동내역, 금액, 단위, 단위_배수, 조정된_금액
(1, 2014, '현금및현금성자산', '대여금 및 수취채권', 1643318.0, '백만원', 1.0, 1643318.0)
(2, 2014, '현금및

# 점검

In [105]:
import sqlite3

# 1. SQLite 데이터베이스에 연결
db_name = 'samsung_audit_report.db'
try:
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    print(f"✅ 데이터베이스 '{db_name}'에 성공적으로 연결되었습니다.")

    # 2. sqlite_master 테이블을 조회하여 모든 테이블 이름 가져오기
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    if tables:
        # 테이블 개수 세기
        table_count = len(tables)
        print(f"\n=== 총 {table_count}개의 테이블이 존재합니다. ===")
        
        # 테이블 목록 출력
        for table in tables:
            print(f"- {table[0]}")
    else:
        print("\n❌ 데이터베이스에 테이블이 존재하지 않습니다.")

except sqlite3.Error as e:
    print(f"❌ 오류: 데이터베이스 연결 또는 조회 중 문제가 발생했습니다. 오류 내용: {e}")

finally:
    # 3. 데이터베이스 연결 종료
    if 'conn' in locals() and conn:
        conn.close()
        print("\n데이터베이스 연결이 정상적으로 종료되었습니다.")

✅ 데이터베이스 'samsung_audit_report.db'에 성공적으로 연결되었습니다.

=== 총 49개의 테이블이 존재합니다. ===
- 재고자산_내역
- sqlite_sequence
- 종속기업_관계기업_공동기업_투자_변동내역
- 종속기업_재무정보
- 유형자산_변동내역
- 무형자산_변동내역
- 관계기업_투자주식_내역
- 관계기업_투자_현황
- 관계기업_재무정보
- Level3_금융상품_변동내역
- 기타자본
- 법인세
- 비용수익
- 자본금_이익잉여금
- 주당이익_배당
- 환율변동위험
- 부채비율
- 공정가치
- 공정가치_민감도분석
- 부문별_보고
- 유동성위험
- 공정가치_수준별
- 거래내역
- 경영진보상
- 지배관계
- 채권채무
- 매각예정분류
- 무상할당_배출권
- 사외적립자산
- 확정급여채무_민감도분석
- 보험수리적_가정
- 충당부채
- 확정급여제도_비용
- 확정급여제도_관련_비용
- 확정급여채무_변동내역
- 확정급여채무_통합
- 매도가능금융자산_변동내역
- 매도가능금융자산_상장주식
- 매출채권_할인내역
- 매출채권과미수금
- 매출채권과미수금_연체일구분
- 매출채권과미수금_충당금변동내역
- 범주별금융상품_금융부채
- 범주별금융상품_금융자산
- 범주별순손익구분_금융부채
- 범주별순손익구분_금융자산
- 사채내역
- 사채상환내역
- 차입금내역

데이터베이스 연결이 정상적으로 종료되었습니다.


CSV 파일 48개 + sqlite_sequence(자동생성)